In [1]:
import mido
import numpy as np
import itertools
import random

MAX_NUM_OF_CHANNELS = 16
DRUM_CHANNEL = 9
MOODS_NAME = ['angry', 'calm', 'happy', 'sad']
MOOD_ANGRY = 0
MOOD_CALM = 1
MOOD_HAPPY = 2
MOOD_SAD = 3

# return true if the message is a note
def is_note(msg):
    return msg.type == 'note_on' or msg.type == 'note_off'

# return true if the message indicates the start of a note
# return false if the message indicates the end of a note
def is_note_on(msg):
    return msg.type == 'note_on' and msg.velocity != 0

def is_drum(msg):
    return msg.channel == DRUM_CHANNEL

# get the time and tick indicating the bars, and their beat and tempo
def get_bars_info(midi_file):
    
    ticks_per_beat = midi_file.ticks_per_beat

    total_tick = 0
    total_time = 0
    rhythm_events = []

    # store the time signature and tempo change events as rhythm_events
    for msg in midi_file:

        # sum up the tick/time
        if msg.time != 0:
            total_tick += mido.second2tick(msg.time, ticks_per_beat, current_tempo)
            total_time += msg.time

        # store meta message of time signature and tempo with current tick
        if msg.is_meta:

            if msg.type == 'time_signature':
                time_signature_quarter = msg.numerator / (msg.denominator / 4)
                rhythm_events.append([total_tick, -1, time_signature_quarter])

            if msg.type == 'set_tempo':
                current_tempo = msg.tempo
                rhythm_events.append([total_tick, current_tempo, -1])
    
    # store the tick of the end of midi
    rhythm_events.append([total_tick, -1, -1])

    cumulative_time = 0
    cumulative_tick = 0
    prev_tick = 0
    current_tempo = -1
    current_ts_quarter = -1
    bars_time = []
    bars_tick = []
    bars_beat = []
    bars_tempo = []

    # append the time/tick at each bar according to 'rhythm_events'
    for rhythm_event in rhythm_events:

        # detect rhythm change, then conclude the bars before the change
        if rhythm_event[0] != prev_tick:

            tick_diff = rhythm_event[0] - prev_tick
            prev_tick = rhythm_event[0]

            # conclude the bar time/tick by dividing the bar length
            # given total tick, and constant beat and tempo, the bars can be separated by dividing evenly
            num_of_bar = tick_diff / ticks_per_beat
            num_of_bar_2 = num_of_bar
            while num_of_bar > 0:
                bar_tick = (num_of_bar_2 - num_of_bar) * ticks_per_beat
                bars_time.append(mido.tick2second(bar_tick, ticks_per_beat, current_tempo) + cumulative_time)
                bars_tick.append(bar_tick + cumulative_tick)
                bars_beat.append(current_ts_quarter)
                bars_tempo.append(current_tempo)
                num_of_bar -= current_ts_quarter            

            cumulative_time += mido.tick2second(tick_diff, ticks_per_beat, current_tempo)
            cumulative_tick += tick_diff

        # update the time signature and time tempo events
        if rhythm_event[1] != -1:
            current_tempo = rhythm_event[1]
        if rhythm_event[2] != -1:
            current_ts_quarter = rhythm_event[2]

    return bars_time, bars_tick, bars_beat, bars_tempo

# get the details of notes in each bar
# i.e. bars_note_info[bar index] = [start tick of the bar, end tick of the bar, bar_note_info]
# i.e. bar_note_info[note index] = [channel index, note no., triggered time (in tick), 1 = 'note_on' / 0 = 'note_off']
def get_bars_note_info(midi_file):

    _, bars_tick, bars_beat, _ = get_bars_info(midi_file)

    ticks_per_beat = midi_file.ticks_per_beat
    current_tempo = -1
    current_ts_quarter = -1

    total_tick = 0
    bar_start_tick = bars_tick.pop(0)
    bar_note_info = []
    bars_note_info = []

    # store the time signature and tempo change events as rhythm_events
    for msg in midi_file:

        # sum up the tick/time
        if msg.time != 0:        
            total_tick += mido.second2tick(msg.time, ticks_per_beat, current_tempo)
            if len(bars_tick) != 0 and total_tick >= bars_tick[0]:

                # find the suitable bar for placing current note
                bars_shift_idx = 0
                while len(bars_tick) > (bars_shift_idx + 1) and total_tick >= bars_tick[bars_shift_idx + 1]:
                    bars_shift_idx += 1

                bars_note_info.append([bar_start_tick, bars_tick[0] - bar_start_tick, bar_note_info, bars_beat[0]])
                bar_note_info = []
                bar_start_tick = bars_tick.pop(0)
                bars_beat.pop(0)

                # put back the shifted (empty) bars
                for pad_bar_idx in range(bars_shift_idx):
                    bars_note_info.append([bar_start_tick, bars_tick[0] - bar_start_tick, bar_note_info, bars_beat[0]])
                    bar_note_info = []
                    bar_start_tick = bars_tick.pop(0)
                    bars_beat.pop(0)


        # store meta message of time signature and tempo with current tick
        if msg.is_meta:

            if msg.type == 'time_signature':
                current_ts_quarter = msg.numerator / (msg.denominator / 4)

            if msg.type == 'set_tempo':
                current_tempo = msg.tempo

        if not msg.is_meta:

            if is_note(msg):

                if is_note_on(msg):
                    bar_note_info.append([msg.channel, msg.note, total_tick, 1, msg.velocity])

                else:
                    bar_note_info.append([msg.channel, msg.note, total_tick, 0, msg.velocity])
    return bars_note_info

# get rhythm pattern in each bar of each channel
# i.e. tracks_rhythm_pattern[channel index, bar index] = [[note1 start_time, note1 duration], [note2 start_time, note2 duration], ...]
def get_rhythm_pattern(bars_note_info):

    MAX_NUM_OF_CHANNELS = 16
    MAX_NUM_OF_NOTE_IN_BAR = 64
    tracks_rhythm_pattern = np.zeros([MAX_NUM_OF_CHANNELS, len(bars_note_info), MAX_NUM_OF_NOTE_IN_BAR, 5]) - 1
    # print(tracks_rhythm_pattern.shape)


    prev_bar_notes = []

    for idx, bar_note_info in enumerate(bars_note_info):

        if len(prev_bar_notes) != 0:
            for prev_bar_note in prev_bar_notes:
                append_idx = 0
                while append_idx < 64:
                    if tracks_rhythm_pattern[prev_bar_note[0], idx, append_idx, 0] == -1:
                        tracks_rhythm_pattern[prev_bar_note[0], idx, append_idx] = [0, prev_bar_note[1], 0, 0, 0]
                        break
                    append_idx += 1
            prev_bar_notes = []

        # print(idx, bar_note_info[2])
        for note in bar_note_info[2]:
            if note[3]:
                append_idx = 0
                while append_idx < 64:
                    if tracks_rhythm_pattern[note[0], idx, append_idx, 0] == -1:
                        tracks_rhythm_pattern[note[0], idx, append_idx, 0] = note[2] - bar_note_info[0]
                        tracks_rhythm_pattern[note[0], idx, append_idx, 1] = -2 - note[1]
                        break
                    append_idx += 1
            else:
                # print(tracks_rhythm_pattern[note[0], idx, :, 1], -2 - note[1])
                relative_idx = np.nonzero(tracks_rhythm_pattern[note[0], idx, :, 1] == -2 - note[1])[0][0]
                tracks_rhythm_pattern[note[0], idx, relative_idx, 1] = note[2] - bar_note_info[0] - tracks_rhythm_pattern[note[0], idx, relative_idx, 0]
                # print(tracks_rhythm_pattern[note[0], idx, relative_idx])

        for channel_idx in range(MAX_NUM_OF_CHANNELS):

            temp_idx = 0

            while temp_idx < MAX_NUM_OF_NOTE_IN_BAR and tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1] != -1:
                if tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1] < -1:
                    prev_bar_notes.append([channel_idx, tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1]])
                    tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1] = bar_note_info[1] - tracks_rhythm_pattern[note[0], idx, relative_idx, 0]
                    # print(tracks_rhythm_pattern[channel_idx, idx, temp_idx])
                tracks_rhythm_pattern[channel_idx, idx, temp_idx] /= bar_note_info[1]
                temp_idx += 1
        # print(prev_bar_notes)
    
    return tracks_rhythm_pattern

def get_midi_info(bars_note_info):
    midi_info = np.empty((len(bars_note_info), MAX_NUM_OF_CHANNELS), dtype=np.ndarray)
    for i in range(midi_info.shape[0]):
        for j in range(midi_info.shape[1]):
            midi_info[i, j] = np.empty((0, 7))


    prev_bar_notes = []

    for idx, bar_note_info in enumerate(bars_note_info):

        if len(prev_bar_notes) != 0:
            for prev_bar_note in prev_bar_notes:
                midi_info[idx, prev_bar_note[0]] = np.vstack((midi_info[idx, prev_bar_note[0]], [prev_bar_note[2], prev_bar_note[3], 0, prev_bar_note[1], 0, 0, 1]))
            prev_bar_notes = []

        for note in bar_note_info[2]:
            if note[3]:
                midi_info[idx, note[0]] = np.vstack((midi_info[idx, note[0]], [note[1], note[4], note[2] - bar_note_info[0], -2 - note[1], 0, 0, 0]))
            else:
                relative_idx = np.nonzero(midi_info[idx, note[0]][:, 3] == -2 - note[1])[0][0]
                midi_info[idx, note[0]][relative_idx, 3] = note[2] - bar_note_info[0] - midi_info[idx, note[0]][relative_idx, 2]

        for channel_idx in range(MAX_NUM_OF_CHANNELS):

            for i, info in enumerate(midi_info[idx, channel_idx]):
                if info[3] < -1:
                    prev_bar_notes.append([channel_idx, info[3], info[0], info[1]])
                    midi_info[idx, channel_idx][i, 3] = bar_note_info[1] - info[2]
                    midi_info[idx, channel_idx][i, 6] = 1
            midi_info[idx, channel_idx][:, 2:4] /= bar_note_info[1]
            midi_info[idx, channel_idx][:, 4:6] = midi_info[idx, channel_idx][:, 2:4] * bar_note_info[3]
    return midi_info

# turn numeric notes into english notes, octave info not preserved
def note_int2char(note):
    return {
        '0': 'C', '1': 'Db', '2': 'D', '3': 'Eb',
        '4': 'E', '5': 'F', '6': 'F#', '7': 'G',
        '8': 'Ab', '9': 'A', '10': 'Bb', '11': 'B'
    }.get(str(note % 12), 'undef')

# define the chords and its relative position
chords = np.empty((0, 12), dtype=np.int8)

chords_entries = np.array([
    # C1
    [6, -1, 0, -4, -4, 0, -1, -5, -1, -1, -3, -3],
    # C5
    [5, -1, 0, -4, -4, 0, -1, 5, -1, -1, -3, -3],
    # C
    [5, -3, 0, -4, 4, 0, -3, 4, -3, 1, -4, -4],
    # Cm
    [5, -3, 0, 4, -4, 0, -3, 4, 1, 0, -4, -4],
    # C7
    [5, -3, 0, -3, 3, 0, -3, 3, 1, 1, 3, -4],
    # CMaj7
    [5, -3, 0, -3, 3, 0, -3, 3, -3, 1, -3, 3],
    # Cmin7
    [5, -3, 0, 3, -3, 0, -3, 3, 1, 0, 3, -1],
    # Cdim
    [5, 0, -3, 4, -3, -3, 4, -4, 0, -3, -3, -3],
], dtype=np.int8)

for chords_entry in chords_entries:
    for i in range(12):
        chords = np.vstack((chords, np.roll(chords_entry, i)))

# name the chord
def name_chord(chord):
    return {
        '0': note_int2char(chord) + '1',
        '1': note_int2char(chord) + '5',
        '2': note_int2char(chord),
        '3': note_int2char(chord) + 'm',
        '4': note_int2char(chord) + '7',
        '5': note_int2char(chord) + 'M7',
        '6': note_int2char(chord) + 'm7',
        '7': note_int2char(chord) + 'dim'
    }.get(str(chord // 12), 'undef')

# chords to key, '[C, Db, D, ..., B, Cm, Dbm, ..., Bbm, Bm]'
chord_to_key = np.empty((0, 36), dtype=np.int8)

chord_to_key_entries = np.array([
    # C note only
    [2, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 2, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0],
    # C power chords
    [2, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0],
    # C Major chords
    [2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
    # C Minor chords
    [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    # C Dominant 7th chords
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    # C Major 7th chords
    [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    # C Minor 7th chords
    [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    # C Diminished chords
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
], dtype=np.int8)

for chord_to_key_entry in chord_to_key_entries:
    for i in range(12):
        chord_to_key = np.vstack((chord_to_key, np.concatenate((np.roll(chord_to_key_entry[0:12], i), np.roll(chord_to_key_entry[12:24], i), np.roll(chord_to_key_entry[24:36], i)))))

# name the key
def name_key(key):
    return {
        '0': note_int2char(key) + ' Major',
        '1': note_int2char(key) + ' minor',
        '2': note_int2char(key) + ' minor (harmonic)',
    }.get(str(key // 12), 'undef')

# get chords of a key
def key_to_chords(key):
    return np.nonzero(np.transpose(chord_to_key)[key])

def get_chords(midi_info):
    chords_predict = np.zeros(midi_info.shape[0], dtype=np.uint8)
    for bar_idx in range(midi_info.shape[0]):
        notes_stat = np.zeros(12, dtype=np.float)
        for channel_idx in range(MAX_NUM_OF_CHANNELS):
            if channel_idx != DRUM_CHANNEL:
                for info in midi_info[bar_idx, channel_idx]:
                    notes_stat[int(info[0] % 12)] += info[3]
        chords_score = np.dot(chords, notes_stat)
        chords_predict[bar_idx] = np.argmax(chords_score)
#         with np.printoptions(precision=3, suppress=True):
#             print(notes_stat)
#             print(["%s: %f"%(note_int2char(idx), note) for idx, note in enumerate(notes_stat) if note > 0])
#             print(name_chord(chords_predict[bar_idx]))
#             print([name_chord(chord) for chord in np.argsort(chords_score)[-10:]])
#             print(chords_score[np.argsort(chords_score)[-10:]])
    return chords_predict

def get_key(chords_predict):
    chords_stat = np.zeros(chords.shape[0], dtype=np.int)
    for chord_predict in chords_predict:
        chords_stat[chord_predict] += 1
    key_stat = np.dot(chords_stat, chord_to_key)
    key_predict = np.argmax(key_stat)
#     with np.printoptions(precision=3, suppress=True):
#         print([name_key(key) for key in np.argsort(key_stat)[-3:]])
#         print(key_stat[np.argsort(key_stat)[-3:]])
    key_acc = np.sum(chords_stat[key_to_chords(key_predict)]) / np.sum(chords_stat)
    return key_predict, key_acc, chords_stat

# keys notes, '[C, Db, D, ..., B, Cm, Dbm, ..., Bbm, Bm]'
keys_notes = np.empty((0, 12), dtype=np.int8)

keys_notes_entries = np.array([
    # C Major
    [2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    # C minor
    [2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0],
    # C minor (harmonic)
    [2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1],
], dtype=np.int8)

for keys_notes_entry in keys_notes_entries:
    for i in range(12):
        keys_notes = np.vstack((keys_notes, np.roll(keys_notes_entry[0:12], i)))

# chords notes, '[C, Db, D, ..., B, Cm, Dbm, ..., Bbm, Bm]'
chords_notes = np.empty((0, 12), dtype=np.int8)

chords_notes_entries = np.array([
    # C1
    [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    # C5
    [2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    # C
    [2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    # Cm
    [2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1],
    # C7
    [2, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0],
    # CMaj7
    [2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    # Cmin7
    [2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0],
    # Cdim
    [2, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
], dtype=np.int8)

for chords_notes_entry in chords_notes_entries:
    for i in range(12):
        chords_notes = np.vstack((chords_notes, np.roll(chords_notes_entry, i)))
        
# chords element, '[C, Db, D, ..., B, Cm, Dbm, ..., Bbm, Bm]'
chords_elements = np.empty((0, 12), dtype=np.int8)

chords_elements_entries = np.array([
    # C1
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    # C5
    [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    # C
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    # Cm
    [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    # C7
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
    # CMaj7
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    # Cmin7
    [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    # Cdim
    [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
], dtype=np.int8)

for chords_elements_entry in chords_elements_entries:
    for i in range(12):
        chords_elements = np.vstack((chords_elements, np.roll(chords_elements_entry, i)))

def is_note_in_chord(note, chord):
    return chords_elements[(int)(chord)][(int)(note) % 12] == 1        

CHANNEL_TYPE_NAME = ['lead', 'pad', 'percussion', 'empty']
CHANNEL_TYPE_LEAD = 0
CHANNEL_TYPE_PAD = 1
CHANNEL_TYPE_PERCUSSION = 2
CHANNEL_TYPE_EMPTY = 3
        
def get_channels_type(midi_info, chords_predict):
    channels_inchord_count = np.zeros((MAX_NUM_OF_CHANNELS))
    channels_all_count = np.zeros((MAX_NUM_OF_CHANNELS))
    channels_chord_acc = np.zeros((MAX_NUM_OF_CHANNELS))
    for bar_idx in range(midi_info.shape[0]):
        for channel_idx in range(MAX_NUM_OF_CHANNELS):
            for note_info in midi_info[bar_idx, channel_idx]:
                if is_note_in_chord(note_info[0], chords_predict[bar_idx]):
                    channels_inchord_count[channel_idx] += note_info[3]
                channels_all_count[channel_idx] += note_info[3]
    
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if channels_all_count[channel_idx] != 0:
            channels_chord_acc[channel_idx] = channels_inchord_count[channel_idx] / channels_all_count[channel_idx]
    
    channels_type = np.zeros((MAX_NUM_OF_CHANNELS), dtype=np.int8)
    inchord_count = 0
    all_count = 0
    chord_acc = 0
    
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if channels_chord_acc[channel_idx] == 0:
            channels_type[channel_idx] = CHANNEL_TYPE_EMPTY
        elif channels_chord_acc[channel_idx] <= 0.5:
            channels_type[channel_idx] = CHANNEL_TYPE_PERCUSSION
        else:
            inchord_count += channels_inchord_count[channel_idx]
            all_count += channels_all_count[channel_idx]
    chord_acc = inchord_count / all_count
    
#     with np.printoptions(precision=3, suppress=True):
#         print("chord accuracy for 16 channels:")
#         print(channels_chord_acc)
    
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if channels_type[channel_idx] == 0:
            if channels_chord_acc[channel_idx] < chord_acc:
                channels_type[channel_idx] = CHANNEL_TYPE_LEAD
            else:
                channels_type[channel_idx] = CHANNEL_TYPE_PAD
    return channels_type

# get each note scores in a bar based on only its note (higher score = more related to the key)
def get_bar_notes_priority(bar_info, key_predict, chord_predict):
    notes_score = keys_notes[key_predict] + chords_notes[chord_predict]
    bar_notes_priority = np.zeros(bar_info.shape[0], dtype=np.uint8)
    for idx, note_info in enumerate(bar_info):
        bar_notes_priority[idx] = notes_score[(int)(note_info[0] % 12)]
    return bar_notes_priority

def get_msg_non_note(midi_file):
    msg_non_note = np.empty((0,2))
    total_tick = 0
    ticks_per_beat = midi_file.ticks_per_beat
    current_tempo = 0

    for msg in midi_file:
        if msg.time != 0:
            total_tick += mido.second2tick(msg.time, ticks_per_beat, current_tempo)

        if msg.is_meta:
            # update tempo for 'second2tick'
            if msg.type == 'set_tempo':
                current_tempo = msg.tempo
            msg_non_note = np.vstack((msg_non_note, np.array([total_tick, msg])))
        else:
            if not is_note(msg):
                msg_non_note = np.vstack((msg_non_note, np.array([total_tick, msg])))
    return msg_non_note

def get_rhythm_stats(midi_info, channels_type):
    rhythm_start_stats = np.empty((MAX_NUM_OF_CHANNELS), dtype=np.object)
    rhythm_duration_stats = np.empty((MAX_NUM_OF_CHANNELS), dtype=np.object)
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if channels_type[channel_idx] != CHANNEL_TYPE_EMPTY:
            rhythm_start_stat = np.empty((0, 2))
            rhythm_duration_stat = np.empty((0, 2))
            for bar_idx in range(midi_info.shape[0]):
                for note_info in midi_info[bar_idx, channel_idx]:
                    if rhythm_start_stat.shape[0] == 0:
                        rhythm_start_stat = np.vstack((rhythm_start_stat, [note_info[4], 1]))
                        rhythm_duration_stat = np.vstack((rhythm_duration_stat, [note_info[5], 1]))
                    else:
                        if note_info[4] in rhythm_start_stat[:, 0]:
                            rhythm_start_stat[np.argwhere(rhythm_start_stat[:, 0] == note_info[4])[0][0], 1] += 1
                        else:
                            rhythm_start_stat = np.vstack((rhythm_start_stat, [note_info[4], 1]))
                        if note_info[5] in rhythm_duration_stat[:, 0]:
                            rhythm_duration_stat[np.argwhere(rhythm_duration_stat[:, 0] == note_info[5])[0][0], 1] += 1
                        else:
                            rhythm_duration_stat = np.vstack((rhythm_duration_stat, [note_info[5], 1]))
            
            rhythm_start_stat_epss = [0.04, 0.02, 0.01, 0.005, 0]
            rhythm_duration_stat_epss = [0.04, 0.02, 0.01, 0.005, 0]
            
            for rhythm_start_stat_eps in rhythm_start_stat_epss:
                rhythm_start_stats[channel_idx] = rhythm_start_stat[rhythm_start_stat[:, 1] >= np.sum(rhythm_start_stat[:, 1]) * rhythm_start_stat_eps]
                if rhythm_start_stats[channel_idx].shape[0] != 0 and rhythm_start_stats[channel_idx].shape[0] / rhythm_start_stat.shape[0] >= 0.75:
                    break
            
            for rhythm_duration_stat_eps in rhythm_duration_stat_epss:
                rhythm_duration_stats[channel_idx] = rhythm_duration_stat[rhythm_duration_stat[:, 1] >= np.sum(rhythm_duration_stat[:, 1]) * rhythm_duration_stat_eps]
                if rhythm_duration_stats[channel_idx].shape[0] != 0 and rhythm_duration_stats[channel_idx].shape[0] / rhythm_duration_stat.shape[0] >= 0.75:
                    break                       
            
    return rhythm_start_stats, rhythm_duration_stats

RHYTHM_UNITS = np.array([
    1,
    1 / 2,
    1 / 3,
    1 / 4,
    1 / 6,
    1 / 8,
    1 / 12,
    1 / 16,
    1 / 24,
    1 / 32,
])

def get_rhythm_recommends(rhythm_start_stats, rhythm_duration_stats):
    rhythm_start_recommends = np.empty((MAX_NUM_OF_CHANNELS), dtype=np.object)
    rhythm_min_units = np.zeros((MAX_NUM_OF_CHANNELS))
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if rhythm_start_stats[channel_idx] is None or rhythm_start_stats[channel_idx].shape[0] == 0:
            continue
        unit = 0
        unit_min = 1
        unit_val = 12345
        for unit in RHYTHM_UNITS:
            if np.sum(np.mod(rhythm_start_stats[channel_idx][:, 0] / unit, 1)) < unit:
                unit_min = -1
                break
            else:
                if np.count_nonzero(np.mod(rhythm_start_stats[channel_idx][:, 0] / unit, 1)) < unit_val:
                    unit_min = unit
                    unit_val = np.count_nonzero(np.mod(rhythm_start_stats[channel_idx][:, 0] / unit, 1))
        if unit_min != -1:
            unit = unit_min
        beat = np.ceil(np.max(rhythm_start_stats[channel_idx][:, 0]))
        rhythm_start_recommends[channel_idx] = np.linspace(0, beat, num=(int)(beat / unit), endpoint=False)
        rhythm_min_units[channel_idx] = unit

    rhythm_duration_recommends = np.empty((MAX_NUM_OF_CHANNELS), dtype=np.object)
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if rhythm_duration_stats[channel_idx] is None or rhythm_duration_stats[channel_idx].shape[0] == 0:
            continue

        offset_min = 100
        for rhythm_duration_stat in rhythm_duration_stats[channel_idx]:
            if rhythm_start_recommends[channel_idx] is None or rhythm_start_recommends[channel_idx].shape[0] == 0:
                continue
            for rhythm_start_recommend in rhythm_start_recommends[channel_idx]:
                if rhythm_start_recommend - rhythm_duration_stat[0] >= 0:
                    if offset_min > (rhythm_start_recommend - rhythm_duration_stat[0]) and (rhythm_start_recommend - rhythm_duration_stat[0]) > 0:
                        offset_min = rhythm_start_recommend - rhythm_duration_stat[0]
                    break
        if offset_min == 100:
            offset_min = 0

        rhythm_duration_recommend = np.array([])
        if rhythm_start_recommends[channel_idx] is not None:
            for rhythm_start_recommend in rhythm_start_recommends[channel_idx]:
                duration_recommend = rhythm_start_recommend / 2 - offset_min
                if duration_recommend > 0:
                    rhythm_duration_recommend = np.hstack((rhythm_duration_recommend, duration_recommend))
                duration_recommend = rhythm_start_recommend - offset_min
                if duration_recommend > 0:
                    rhythm_duration_recommend = np.hstack((rhythm_duration_recommend, duration_recommend))
        rhythm_duration_recommends[channel_idx] = rhythm_duration_recommend
    return rhythm_start_recommends, rhythm_duration_recommends, rhythm_min_units

def rhythm_change(nMood, midi_info, midi_chord, midi_key, bars_beat, channels_type, rhythm_start_recommends, rhythm_duration_recommends, rhythm_min_units):
    for bar in range(midi_info.shape[0]):
        for channel in range(midi_info.shape[1]):
            if channels_type[channel] == CHANNEL_TYPE_EMPTY:
                continue
            if nMood == MOOD_ANGRY: #angry
                if channels_type[channel] != CHANNEL_TYPE_PERCUSSION:
                    midi_info[bar, channel] = moreNote("high", midi_info, bar, channel, bars_beat[bar], rhythm_start_recommends, rhythm_duration_recommends, rhythm_min_units)
                    midi_info[bar, channel] = thicker(midi_info, midi_chord, bar, channel, bars_beat[bar])
            if nMood == MOOD_CALM: #calm
                midi_info[bar, channel] = lessNote("high", midi_info, midi_chord, bar, channel)
                midi_info[bar, channel] = lessSyn(midi_info, bar, channel)
                midi_info[bar, channel] = thinner(midi_info, bar, channel, midi_chord, midi_key)
            if nMood == MOOD_HAPPY and channel != DRUM_CHANNEL: #happy
                if channels_type[channel] == CHANNEL_TYPE_LEAD:
                    midi_info[bar, channel] = moreSyn("low", midi_info, bar, channel, bars_beat[bar], rhythm_start_recommends, rhythm_duration_recommends)
                else:
                    midi_info[bar, channel] = moreSyn("high", midi_info, bar, channel, bars_beat[bar], rhythm_start_recommends, rhythm_duration_recommends)
                if channels_type[channel] != CHANNEL_TYPE_PERCUSSION:
                    midi_info[bar, channel] = moreNote("low", midi_info, bar, channel, bars_beat[bar], rhythm_start_recommends, rhythm_duration_recommends, rhythm_min_units)
                if channels_type[channel] != CHANNEL_TYPE_PERCUSSION:
                    midi_info[bar, channel] = thicker(midi_info, midi_chord, bar, channel, bars_beat[bar])
            if nMood == MOOD_SAD: #sad
                midi_info[bar, channel] = lessNote("low", midi_info, midi_chord, bar, channel)
                midi_info[bar, channel] = lessSyn(midi_info, bar, channel)
            midi_info[bar, channel] = extend(midi_info, bar, channel, bars_beat[bar], rhythm_duration_recommends, rhythm_min_units)

def check_on_beat(degree, start_time, no_of_beat): #checked
    each_beat = 1 / no_of_beat
    on_beat = np.array([0])
    for a in range(1, no_of_beat):
        np.append(on_beat, a * each_beat)
        
    for b in on_beat:
        if ((abs(start_time - b) <= 0.15) and degree == "low") or ((abs(start_time - b) <= 0.35) and degree == "high") :
            return True
    return False

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def moreNote(degree, midi_info, bar, channel, beat, rhythm_start_recommends, rhythm_duration_recommends, rhythm_min_units): #checked
    bar_channel_info = midi_info[bar, channel]
    
    limit = (int)(round(bar_channel_info.shape[0] * 0.3))
    
    for note in range(bar_channel_info.shape[0]):
        note_info = bar_channel_info[note]
        mod3 = random.randint(0, 2)
        mod5 = random.randint(0, 4)
        if (((note + 1) % 3 == mod3 and degree == "high") or ((note + 1) % 5 == mod5 and degree == "low")) and limit > 0 and note_info[6] == 0 and note_info[5] >= rhythm_min_units[channel] * 2:
            note_info[5] /= 2
            note_info[5] = find_nearest(rhythm_duration_recommends[channel], note_info[5])
            note_info[3] = note_info[5] / beat
            
            new_note_start = find_nearest(rhythm_start_recommends[channel], note_info[4] + note_info[5])
            
            newNote = np.array([-1, int(note_info[1] * 0.5), new_note_start / beat, note_info[3], new_note_start, note_info[5], 0])
            bar_channel_info = np.insert(bar_channel_info, note + 1, newNote, 0)
            limit -= 1
    return bar_channel_info

def lessNote(degree, midi_info, midi_chord, bar, channel): #checked
    bar_channel_info = midi_info[bar, channel]
    note_priority = get_bar_notes_priority(bar_channel_info, key_predict, midi_chord[bar])
    priority_sort = np.argsort(note_priority)
    cut_quota = (int)(round(bar_channel_info.shape[0] * 0.5))
    limit = cut_quota
    cut_list = np.empty(0)
    for note in reversed(range(bar_channel_info.shape[0])):
        note_info = bar_channel_info[note]

        if note_info[3] != 0 and not priority_sort is None:
            if not check_on_beat("low", note_info[2], (int)(round(note_info[5] / note_info[3]))) and (np.where(priority_sort == note)[0] <= cut_quota):
                if note - 1 > 0:
                    dur = note_info[5]
                    wDur = round(bar_channel_info[note - 1][5] / bar_channel_info[note - 1][3])
                    bar_channel_info[note - 1][5] += dur
                    bar_channel_info[note - 1][3] = bar_channel_info[note - 1][5] / wDur
                    cut_list = np.append(cut_list, note)
    cut_list = np.flip(cut_list)
    
    temp_a = cut_list
    temp_b = np.linspace(0, bar_channel_info.shape[0], num=bar_channel_info.shape[0], endpoint=False)
    temp_c = np.setdiff1d(temp_b, temp_a)
    temp_c = temp_c.astype(int)
    if temp_c.shape[0] != 0:
        bar_channel_info = bar_channel_info[temp_c]
#         print(bar_channel_info[temp_c])
    
#     for a in range(cut_list.shape[0]):
#         for b in range(a, cut_list.shape[0]):
#             if cut_list[b] < cut_list[a]:
#                 cut_list[a] -= 1
#     for c in range(cut_list.shape[0]):
#         if bar_channel_info[c, 6] == 0 and limit > 0:
#             bar_channel_info = np.delete(bar_channel_info, c, 0)
#             limit -= 1
    return bar_channel_info

def moreSyn(degree, midi_info, bar, channel, beat, rhythm_start_recommends, rhythm_duration_recommends): #checked
    bar_channel_info = midi_info[bar, channel]
    for note in range(bar_channel_info.shape[0]):
        note_info = bar_channel_info[note]
        if note_info[3] != 0 and note_info[6] == 0:
            if check_on_beat(degree, note_info[2], int(round(note_info[5] / note_info[3]))):
#                 at1 = bar_channel_info[note][3] / 2
#                 at2 = bar_channel_info[note][5] / 2
#                 bar_channel_info[note][2] += at1
#                 bar_channel_info[note][3] = at1
#                 bar_channel_info[note][4] += at2
#                 bar_channel_info[note][5] = at2
                pad_amount = bar_channel_info[note][5] / 2
                if pad_amount >= 0.25 and pad_amount <= 0.5:
                    bar_channel_info[note][4] = find_nearest(rhythm_start_recommends[channel], bar_channel_info[note][4] + pad_amount)
                    bar_channel_info[note][5] = find_nearest(rhythm_duration_recommends[channel], pad_amount)
                    bar_channel_info[note][2] = bar_channel_info[note][4] / beat
                    bar_channel_info[note][3] = bar_channel_info[note][5] / beat
    return bar_channel_info
    
def lessSyn(midi_info, bar, channel): #checked
    bar_channel_info = midi_info[bar, channel]
    if bar_channel_info.shape[0] > 0:
        cut_count = 0.3 * bar_channel_info.shape[0]
        for t in range(bar_channel_info.shape[0]):
            if bar_channel_info[t][3] == 0:
                break
            no_of_beat = int(round(bar_channel_info[t][5] / bar_channel_info[t][3]))
            cut_list = np.array([1, 1.5, 2, 3, 4])
            l0 = np.zeros(1)
            l1 = np.zeros(1)
            l2 = np.zeros(1)
            l3 = np.zeros(1)
            l4 = np.zeros(1)

            for a in range(cut_list.shape[0]):
                temp = 1 / (no_of_beat * cut_list[a])
                temp_2 = temp
                while temp_2 < 0.9999:
                    if a == 0:
                        l0 = np.append(l0, temp_2)
                    if a == 1:
                        l1 = np.append(l1, temp_2)
                    if a == 2:
                        l2 = np.append(l2, temp_2)
                    if a == 3:
                        l3 = np.append(l3, temp_2)
                    if a == 4:
                        l4 = np.append(l4, temp_2)
                    temp_2 += temp
                    
#             length_units = np.empty((0,), dtype=np.object)
#             length_units = np.append(length_units, l0, axis=0)
#             length_units = np.append(length_units, l1, axis=0)
#             length_units = np.append(length_units, l2, axis=0)
#             length_units = np.append(length_units, l3, axis=0)
#             length_units = np.append(length_units, l4, axis=0)
            length_units = np.array([
                l0,
                l1,
                l2,
                l3,
                l4
            ])
            priority_list = np.zeros(bar_channel_info.shape[0])
            LENGTH_UNIT_EPS = 0.05

            for note_idx in range(priority_list.shape[0]):
                for unit_idx in range(length_units.shape[0]):
                    for u in range(length_units[unit_idx].shape[0]):
                        if abs(bar_channel_info[note_idx][2] - u) < LENGTH_UNIT_EPS:
                            priority_list[note_idx] = cut_list.shape[0] - unit_idx
                            break
                    if priority_list[note_idx] != 0:
                        break
                        
            priority_list_argsort = np.argsort(priority_list)
            note_idx = np.empty(0)
            target_val = np.empty(0)
            
            for edit_idx in range((int)(cut_count)):
                note_idx = np.append(note_idx, priority_list_argsort[edit_idx])
                priority = priority_list[priority_list_argsort[edit_idx]]
                
#                 level = 2
#                 if priority == 5:
#                     level = 0
#                 elif priority == 4:
#                     level = 1
#                 elif priority == 3:
#                     level = 0
#                 elif priority == 2:
#                     level = 1
#                 elif priority == 1:
#                     level = 3
                level = 0
                
                for u in range(length_units[level].shape[0]):
                    if bar_channel_info[priority_list_argsort[edit_idx], 2] < length_units[level][u]:
                        target_val = np.append(target_val, length_units[level][u - 1])
                        break
                
                if target_val.shape[0] != note_idx.shape[0]:
                    target_val = np.append(target_val, length_units[level][-1])
            
            if note_idx.shape[0] > 0:
                for j in range(note_idx.shape[0]):
                    bar_channel_info[(int)(note_idx[j])][2] = target_val[j]
                    bar_channel_info[(int)(note_idx[j])][3] += (bar_channel_info[(int)(note_idx[j])][2] - target_val[j])
                    bar_channel_info[(int)(note_idx[j])][4] = bar_channel_info[(int)(note_idx[j])][2] * no_of_beat
                    bar_channel_info[(int)(note_idx[j])][5] = bar_channel_info[(int)(note_idx[j])][3] * no_of_beat

    return bar_channel_info
    
def get_bar_note_position(bar_channel_info, no_of_beat):
    cut_list = np.array([1, 1.5, 2, 3, 4])
    l0 = np.zeros(1)
    l1 = np.zeros(1)
    l2 = np.zeros(1)
    l3 = np.zeros(1)
    l4 = np.zeros(1)

    for a in range(cut_list.shape[0]):
        temp = 1 / (no_of_beat * cut_list[a])
        temp_2 = temp
        while temp_2 < 0.9999:
            if a == 0:
                l0 = np.append(l0, temp_2)
            if a == 1:
                l1 = np.append(l1, temp_2)
            if a == 2:
                l2 = np.append(l2, temp_2)
            if a == 3:
                l3 = np.append(l3, temp_2)
            if a == 4:
                l4 = np.append(l4, temp_2)
            temp_2 += temp

    length_units = np.array([l0, l1, l2, l3, l4])
    LENGTH_UNIT_EPS = 0.01
    
    priority_list = np.zeros(bar_channel_info.shape[0])
    priority_list += 5

    for note_idx in range(bar_channel_info.shape[0]):
        for unit_idx in range(length_units.shape[0]):
            for u in range(length_units[unit_idx].shape[0]):
                if abs(bar_channel_info[note_idx][2] - length_units[unit_idx][u]) < LENGTH_UNIT_EPS:
                    priority_list[note_idx] = unit_idx
                    break
            if priority_list[note_idx] != 5:
                break
    return priority_list
    
def thicker(midi_info, midi_chord, bar, channel, beat): #checked
    bar_channel_info = midi_info[bar, channel]
    priority_list = get_bar_note_position(bar_channel_info, beat)
    if bar_channel_info.shape[0] > 0:
        note_pad = 0
        for note_idx in range(bar_channel_info.shape[0]):
            note_info = bar_channel_info[note_idx + note_pad]
            new_note = np.array([-1, note_info[1] * 0.7, note_info[2], note_info[3], note_info[4], note_info[5], 0])
            if priority_list[note_idx] == 0:
                bar_channel_info = np.insert(bar_channel_info, note_idx + 1 + note_pad, np.copy(new_note), 0)
                bar_channel_info = np.insert(bar_channel_info, note_idx + 2 + note_pad, np.copy(new_note), 0)
                note_pad += 2
            elif priority_list[note_idx] == 2:
                bar_channel_info = np.insert(bar_channel_info, note_idx + 1 + note_pad, np.copy(new_note), 0)
                note_pad += 1
                
#         bar_chord = midi_chord[bar]
#         newNote = np.zeros(1)
#         flag = 0
#         for note in range(bar_channel_info.shape[0]):
#             note_info = bar_channel_info[note]
#             if is_note_in_chord(note_info[0], bar_chord):
#                 for a in range(1, 12):
#                     if note_info[0] > 12 and is_note_in_chord(note_info[0] - a, bar_chord):
#                         newNote = np.array([note_info[0] - a, note_info[1], note_info[2], note_info[3], note_info[4], note_info[5], 0])
# #                         newNote = np.array([-1, note_info[1], note_info[2], note_info[3], note_info[4], note_info[5], 0])
#                         flag = 1
#             if flag == 1:
#                 bar_channel_info = np.insert(bar_channel_info, note + 1, newNote, 0)
#     with np.printoptions(precision=3, suppress=True):
#         print(bar_channel_info)
    return bar_channel_info

def thinner(midi_info, bar, channel, midi_chord, midi_key): #checked
    bar_channel_info = midi_info[bar, channel]
    bar_notes_priority = get_bar_notes_priority(bar_channel_info, midi_key, midi_chord[bar])
    
    cut_list = np.empty(0)
    for note1 in range(bar_channel_info.shape[0]):
        for note2 in range(note1, bar_channel_info.shape[0]):
            if bar_channel_info[note2][2] == bar_channel_info[note1][2] and (not note2 in cut_list) and bar_channel_info[note2, 6] == 0:
                if bar_notes_priority[note1] > bar_notes_priority[note2]:
                    cut_list = np.append(cut_list, note2)
                elif bar_notes_priority[note1] < bar_notes_priority[note2]:
                    cut_list = np.append(cut_list, note1)
                    break
#             if note2 > note1 and bar_channel_info[note2][2] == bar_channel_info[note1][2] and (not note2 in cut_list) and bar_channel_info[note2, 6] == 0:
#                 cut_list = np.append(cut_list, note2)
    for a in range(cut_list.shape[0]):
        for b in range(cut_list.shape[0]):
            if b > a and cut_list[a] < cut_list[b]:
                cut_list[b] -= 1
    for c in range(cut_list.shape[0]):
        bar_channel_info = np.delete(bar_channel_info, cut_list[c], 0)
    return bar_channel_info

def extend(midi_info, bar, channel, beat, rhythm_duration_recommends, rhythm_min_units):
    bar_channel_info = midi_info[bar, channel]
    if bar_channel_info.shape[0] == 0:
        return bar_channel_info
    
    rhythm_min_duration = find_nearest(rhythm_duration_recommends[channel], rhythm_min_units[channel])
    for note_idx in range(bar_channel_info.shape[0]):
        if bar_channel_info[note_idx, 5] < rhythm_min_units[channel] and bar_channel_info[note_idx, 6] == 0:
            bar_channel_info[note_idx, 5] = rhythm_min_duration
            bar_channel_info[note_idx, 3] = bar_channel_info[note_idx, 5] / beat          
        
    return bar_channel_info
    

# add pitch to notes with -1 pitch
# analysis by "median note over bar", "key" and "bar chord"
def bar_add_pitch(bar_info, key_predict, chord_predict):
    avg_pitch = np.median([note for note in bar_info[:, 0] if note != -1])
    notes_score = keys_notes[key_predict] + chords_notes[chord_predict]
#     print("notes score:\t%s" % notes_score)

    SEARCH_PITCH_RANGE = 12
    min_pitch = (int)(np.max((0, np.min((np.floor(avg_pitch - SEARCH_PITCH_RANGE), np.min([note for note in bar_info[:, 0] if note != -1]))))))
    max_pitch = (int)(np.min((127, np.max((np.floor(avg_pitch + SEARCH_PITCH_RANGE), np.max(bar_info[:, 0]))))))

    notes_priority = np.zeros(max_pitch - min_pitch + 1)
    for i in range(0, max_pitch - min_pitch + 1):
        notes_priority[i] = 1 / (np.abs(avg_pitch - (i + min_pitch)) + SEARCH_PITCH_RANGE) * notes_score[(i + min_pitch) % 12]

#     print(notes_priority)
        
    avail_notes = np.argsort(-notes_priority) + min_pitch
#     print(avail_notes)

    for empty_idx in np.argwhere(bar_info[:, 0] == -1).flatten():
        notes_on = []
        for note_info in [note_info for note_info in bar_info if note_info[0] != -1]:
            bool_overlap = not(note_info[2] >= (bar_info[empty_idx, 2] + bar_info[empty_idx, 3]) or (note_info[2] + note_info[3]) <= bar_info[empty_idx, 2])
            if (note_info[0] not in notes_on and bool_overlap):
                notes_on.append(note_info[0])
        i = 0
#         print(notes_on)
        while (avail_notes[i] in notes_on):
            i += 1
            if i == avail_notes.shape[0]:
                i = 0
                break
        bar_info[empty_idx, 0] = avail_notes[i]
        
    return bar_info
    
def add_pitch(midi_info, mood, key_predict, chords_predict):
    if mood == MOOD_ANGRY or mood == MOOD_HAPPY:
        for bar_idx in range(midi_info.shape[0]):
            for channel_idx in range(MAX_NUM_OF_CHANNELS):
                if (-1 in midi_info[bar_idx, channel_idx][:, 0]):
                    midi_info[bar_idx, channel_idx] = bar_add_pitch(midi_info[bar_idx, channel_idx], key_predict, chords_predict[bar_idx])


# get the overall statistic of dynamics in each channel ([min, max, median])
def get_dynamics_info(midi_info):
    dynamics_info = np.zeros((MAX_NUM_OF_CHANNELS, 4), dtype=np.int16)
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        notes_dynamics = []
        for bar_idx in range(midi_info.shape[0]):
            if midi_info[bar_idx, channel_idx] is None:
                continue
            for bar_info in midi_info[bar_idx, channel_idx]:
                notes_dynamics.append(bar_info[1])
        if len(notes_dynamics) > 0:
            dynamics_info[channel_idx] = [np.min(notes_dynamics), np.max(notes_dynamics), np.median(notes_dynamics), np.std(notes_dynamics)]
    return dynamics_info

def get_dynamics_lookup_table(dynamics_info, wide_factor, avg_addition_boost, avg_multiply_boost):
    dynamics_lookup_table = np.zeros((MAX_NUM_OF_CHANNELS, 128), dtype=np.int16)
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        dmin = dynamics_info[channel_idx, 0]
        dmax = dynamics_info[channel_idx, 1]
        dmed = dynamics_info[channel_idx, 2]
        for i in range(dmin, dmax + 1):
            note_velocity = 0
            if i < dmed:
                note_velocity = (int)(dmed - (dmed - dmin) * np.power((dmed - i) / (dmed - dmin), wide_factor[channel_idx]) * avg_multiply_boost[channel_idx])
            else:
                if dmax == dmed:
                    note_velocity = i
                else:
                    note_velocity = (int)(dmed - (dmed - dmax) * np.power((dmed - i) / (dmed - dmax), wide_factor[channel_idx]) * avg_multiply_boost[channel_idx])
            dynamics_lookup_table[channel_idx, i] = np.max((0, np.min((127, note_velocity + avg_addition_boost[channel_idx]))))
        if dmin != 0:
            for i in range(0, dmin):
                dynamics_lookup_table[channel_idx, i] = (int)(dynamics_lookup_table[channel_idx, dmin]  * i / dmin)
        if dmax != 127:
            for i in range(dmax, 128):
                dynamics_lookup_table[channel_idx, i] = (int)(dynamics_lookup_table[channel_idx, dmax] + (127 - dynamics_lookup_table[channel_idx, dmax])  * (i - dmax) / (127 - dmax))
#     for i in range(MAX_NUM_OF_CHANNELS):
#         print(dynamics_lookup_table[i])
    return dynamics_lookup_table

def get_dynamics_angry_factors(dynamics_info, wide_para_1=0.3, wide_para_2=0.1, addition_para_1=10, multiply_para_1=0.2, multiply_para_2=0):
    dynaics_angry_factors = np.zeros((MAX_NUM_OF_CHANNELS, 3))
    dynaics_angry_factors[:, 0] = 1 - 1 / (dynamics_info[:, 3] * wide_para_1 + wide_para_2 + 1)
    dynaics_angry_factors[:, 1] = np.repeat(addition_para_1, MAX_NUM_OF_CHANNELS)
    dynaics_angry_factors[:, 2] = 1 + multiply_para_1 / (dynamics_info[:, 3] + multiply_para_2 + 1)
    return dynaics_angry_factors

def get_dynamics_calm_factors(dynamics_info, wide_para_1=0.8, addition_para_1=-10, multiply_para_1=0.2, multiply_para_2=0.5):
    dynaics_calm_factors = np.zeros((MAX_NUM_OF_CHANNELS, 3))
    dynaics_calm_factors[:, 0] = 1 + dynamics_info[:, 3] * wide_para_1
    dynaics_calm_factors[:, 1] = np.repeat(addition_para_1, MAX_NUM_OF_CHANNELS)
    dynaics_calm_factors[:, 2] = 1 / (dynamics_info[:, 3] * multiply_para_1 + 1) * (1 - multiply_para_2) + multiply_para_2
    return dynaics_calm_factors
    
def get_dynamics_happy_factors(dynamics_info, wide_para_1=5, wide_para_2=0.8, wide_para_3=0.5, addition_para_1=0, multiply_para_1=0.2, multiply_para_2=0):
    dynamics_happy_factors = np.zeros((MAX_NUM_OF_CHANNELS, 3))
    for i in range(MAX_NUM_OF_CHANNELS):
        if dynamics_info[i, 3] < wide_para_1:
            dynamics_happy_factors[i, 0] = 1 - wide_para_2 * np.power((wide_para_1 - dynamics_info[i, 3]) / wide_para_1, wide_para_2)
        else:
            dynamics_happy_factors[i, 0] = (1 - 1 / ((dynamics_info[i, 3] - wide_para_1) * wide_para_3 + 1)) * wide_para_2 + 1
    dynamics_happy_factors[:, 0] = 1 - 1 / (dynamics_info[:, 3] * wide_para_1 + wide_para_2 + 1)
    dynamics_happy_factors[:, 1] = np.repeat(addition_para_1, MAX_NUM_OF_CHANNELS)
    dynamics_happy_factors[:, 2] = 2 / (dynamics_info[:, 3] + 5) + 0.8
    return dynamics_happy_factors

def dynamics_angry(bar_info, dynamic_info, angry_dynamic_lookup_table, bar_notes_priority):
    if len(bar_info) == 0:
        return np.array([])    
    if np.min(bar_info[:,1]) != np.max(bar_info[:,1]) or len(bar_info) == 1:
        return angry_dynamic_lookup_table[[(int)(velocity) for velocity in bar_info[:, 1]]]
    else:
        dynamics_new = np.array([
            np.round(dynamic_info[0] + (dynamic_info[1] - dynamic_info[0]) / 5),
            np.round(dynamic_info[0] + (dynamic_info[1] - dynamic_info[0]) / 5 * 2),
            np.round(dynamic_info[0] + (dynamic_info[1] - dynamic_info[0]) / 5 * 3),
            np.round(dynamic_info[0] + (dynamic_info[1] - dynamic_info[0]) / 5 * 4),
            np.round(dynamic_info[0] + (dynamic_info[1] - dynamic_info[0]) / 5 * 4)
        ], dtype=np.int8)
#         print(dynamics_new)
        return angry_dynamic_lookup_table[dynamics_new[bar_notes_priority]]

def dynamics_calm(bar_info, calm_dynamic_lookup_table):
    return calm_dynamic_lookup_table[[(int)(velocity) for velocity in bar_info[:, 1]]]

def dynamics_happy(bar_info, happy_dynamic_lookup_table):
    return happy_dynamic_lookup_table[[(int)(velocity) for velocity in bar_info[:, 1]]]

def dynamics_sad_lead(bar_info, dynamic_info):
    if bar_info.shape[0] == 0:
        return []
    notes_velocity = np.zeros((bar_info.shape[0], ), dtype=np.int8)
    notes_velocity_ratio = np.zeros((bar_info.shape[0], ), dtype=np.float_)
    
    bar_med_note = np.median(bar_info[:, 1])
    for i in range(bar_info.shape[0]):
        if i == 0:
            notes_velocity_ratio[i] = 0
        else:
            notes_velocity_ratio[i] = (bar_info[i, 0] - bar_info[i - 1, 0]) / bar_info[i, 0]
    max_ratio = np.max(notes_velocity_ratio)
    min_ratio = np.min(notes_velocity_ratio)
    for i in range(bar_info.shape[0]):
        if notes_velocity_ratio[i] < 0:
            if min_ratio == 0:
                continue
            notes_velocity[i] = (int)(bar_med_note - notes_velocity_ratio[i] * (bar_med_note - dynamic_info[0]) / min_ratio)
        else:
            if max_ratio == 0:
                continue
            notes_velocity[i] = (int)(bar_med_note + notes_velocity_ratio[i] * (dynamic_info[1] - bar_med_note) / max_ratio)
    return notes_velocity
        
def dynamics_sad_pad(bar_info, dynamic_info):
    bar_med_note = np.median(bar_info[:, 1])
    notes_velocity = np.array((1 - bar_info[:, 2]) * (bar_med_note - dynamic_info[0]) + dynamic_info[0], dtype=np.int8)
    return notes_velocity

def change_dynamics(midi_info, mood, channels_type):
    dynamics_info = get_dynamics_info(midi_info)

    if mood == 0:
        dynamics_factors = get_dynamics_angry_factors(dynamics_info)
        dynamics_lookup_table = get_dynamics_lookup_table(dynamics_info, wide_factor=dynamics_factors[:, 0], avg_addition_boost=dynamics_factors[:, 1], avg_multiply_boost=dynamics_factors[:, 2])
        for bar_idx in range(midi_info.shape[0]):
            for channel_idx in range(MAX_NUM_OF_CHANNELS):
                bar_notes_priority = get_bar_notes_priority(midi_info[bar_idx, channel_idx], key_predict, chords_predict[bar_idx])
                new_dynamics = dynamics_angry(midi_info[bar_idx, channel_idx], dynamics_info[channel_idx], dynamics_lookup_table[channel_idx], bar_notes_priority)
                if len(new_dynamics) != 0:
                    midi_info[bar_idx, channel_idx][:, 1] = new_dynamics
    elif mood == 1:
        dynamics_factors = get_dynamics_calm_factors(dynamics_info)
        dynamics_lookup_table = get_dynamics_lookup_table(dynamics_info, wide_factor=dynamics_factors[:, 0], avg_addition_boost=dynamics_factors[:, 1], avg_multiply_boost=dynamics_factors[:, 2])
        dynamics_factors_percussion = get_dynamics_calm_factors(dynamics_info, addition_para_1=-50)
        dynamics_lookup_table_percussion = get_dynamics_lookup_table(dynamics_info, wide_factor=dynamics_factors_percussion[:, 0], avg_addition_boost=dynamics_factors_percussion[:, 1], avg_multiply_boost=dynamics_factors_percussion[:, 2])
        for bar_idx in range(midi_info.shape[0]):
            for channel_idx in range(MAX_NUM_OF_CHANNELS):
                new_dynamics = None
                if channels_type[channel_idx] != CHANNEL_TYPE_PERCUSSION:
                    new_dynamics = dynamics_calm(midi_info[bar_idx, channel_idx], dynamics_lookup_table[channel_idx])
                else:
                    new_dynamics = dynamics_calm(midi_info[bar_idx, channel_idx], dynamics_lookup_table_percussion[channel_idx])
                    
                if len(new_dynamics) != 0:
                    midi_info[bar_idx, channel_idx][:, 1] = new_dynamics
    elif mood == 2:
        dynamics_factors = get_dynamics_happy_factors(dynamics_info)
        dynamics_lookup_table = get_dynamics_lookup_table(dynamics_info, wide_factor=dynamics_factors[:, 0], avg_addition_boost=dynamics_factors[:, 1], avg_multiply_boost=dynamics_factors[:, 2])
        for bar_idx in range(midi_info.shape[0]):
            for channel_idx in range(MAX_NUM_OF_CHANNELS):
                new_dynamics = dynamics_happy(midi_info[bar_idx, channel_idx], dynamics_lookup_table[channel_idx])
                if len(new_dynamics) != 0:
                    midi_info[bar_idx, channel_idx][:, 1] = new_dynamics
    elif mood == 3:
        for channel_idx in range(MAX_NUM_OF_CHANNELS):
            if channels_type[channel_idx] == CHANNEL_TYPE_EMPTY:
                continue
            elif channels_type[channel_idx] == CHANNEL_TYPE_LEAD:
                for bar_idx in range(midi_info.shape[0]):
                    new_dynamics = dynamics_sad_lead(midi_info[bar_idx, channel_idx], dynamics_info[channel_idx])
                    if len(new_dynamics) != 0:
                        midi_info[bar_idx, channel_idx][:, 1] = new_dynamics
            else:
                for bar_idx in range(midi_info.shape[0]):
                    new_dynamics = dynamics_sad_pad(midi_info[bar_idx, channel_idx], dynamics_info[channel_idx])
                    if len(new_dynamics) != 0:
                        midi_info[bar_idx, channel_idx][:, 1] = new_dynamics
                
            
        
def get_module_vector(prev_key, new_key):
    module_vector = np.zeros((128), dtype=np.int8)
    
    tonic_note = prev_key % 12
    prev_key_type = prev_key // 12
    new_key_type = new_key // 12
    
    if prev_key_type == 0:
        # Major to natural minor
        if new_key_type == 1:                    
            range_chain = itertools.chain(range(tonic_note - 8, 128, 12), range(tonic_note - 3, 128, 12), range(tonic_note - 1, 128, 12))
            for i in range_chain:
                if i >= 0:
                    module_vector[i] -= 1
        # Major to harmonic minor
        if new_key_type == 2:                    
            range_chain = itertools.chain(range(tonic_note - 8, 128, 12), range(tonic_note - 3, 128, 12))
            for i in range_chain:
                if i >= 0:
                    module_vector[i] -= 1
                    
    if prev_key_type == 1:
        # natural minor to Major
        if new_key_type == 0:
            range_chain = itertools.chain(range(tonic_note - 9, 128, 12), range(tonic_note - 4, 128, 12), range(tonic_note - 2, 128, 12))
            for i in range_chain:
                if i >= 0:
                    module_vector[i] += 1
        # natural minor to harmonic minor
        if new_key_type == 2:
            for i in range(tonic_note - 2, 128, 12):
                if i >= 0:
                    module_vector[i] += 1
        # harmonic minor to natural minor
        if new_key_type == 1:
            for i in range(tonic_note - 1, 128, 12):
                if i >= 0:
                    module_vector[i] -= 1
                    
    # change tonic key
    module_vector += ((new_key % 12 - prev_key % 12) + 6) % 12 - 6
                        
    return module_vector

CIRCLE_OF_FIFTH = np.array([0, 7, 2, 9, 4, 11, 6, 1, 8, 3, 10, 5], dtype=np.int8)
MOOD_MODULE_TYPE = np.array([
    [1, 1, 2],
    [0, 1, 1],
    [0, 0, 0],
    [2, 2, 2]
])
MOOD_MODULE_TONIC = np.array([-1, 8, 4, 0], dtype=np.int8)

def get_key_to_module(prev_key, mood):
    MAX_MOOD_DIST = 3
    prev_key_tonic = prev_key % 12
    prev_key_type = prev_key // 12
    new_key_type = MOOD_MODULE_TYPE[mood, prev_key_type]
    
    if mood == MOOD_ANGRY:
        return new_key_type * 12 + prev_key_tonic
    else:
        minor_shift = 0
        if new_key_type != 0:
            minor_shift = 3
        circle_idx = np.where(CIRCLE_OF_FIFTH == (prev_key_tonic + minor_shift) % 12)[0][0]
        
        mood_direction = (MOOD_MODULE_TONIC[mood] - circle_idx + 12) % 12 <= (circle_idx - MOOD_MODULE_TONIC[mood] + 12) % 12
        mood_move = -1
        if mood_direction:
            mood_move = 1
                
        tonic_dists = np.array([[CIRCLE_OF_FIFTH[(circle_idx + minor_shift) % 12] - prev_key_tonic, circle_idx]])
        for i in range(MAX_MOOD_DIST):
            if circle_idx == MOOD_MODULE_TONIC[mood]:
                break
            circle_idx = (circle_idx + mood_move) % 12
            tonic_dists = np.vstack((tonic_dists, [CIRCLE_OF_FIFTH[(circle_idx + minor_shift) % 12] - prev_key_tonic, circle_idx]))
            
        for i in range(tonic_dists.shape[0]):
            if tonic_dists[i, 0] < -6:
                tonic_dists[i, 0] += 12
            elif tonic_dists[i, 0] >= 6:
                tonic_dists[i, 0] -= 12
        
        if mood == MOOD_CALM or mood == MOOD_SAD:
            circle_idx = tonic_dists[np.argmin(tonic_dists[:, 0]), 1]
        elif mood == MOOD_HAPPY:
            circle_idx = tonic_dists[np.argmax(tonic_dists[:, 0]), 1]
                    
        return new_key_type * 12 + CIRCLE_OF_FIFTH[(circle_idx + minor_shift) % 12]

def module_key(midi_info, module_vector):
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if channel_idx == DRUM_CHANNEL:
            continue
        for bar_idx in range(midi_info.shape[0]):
            for info_idx in range(midi_info[bar_idx, channel_idx].shape[0]):
                midi_info[bar_idx, channel_idx][info_idx, 0] += module_vector[(int)(midi_info[bar_idx, channel_idx][info_idx, 0])]
    return midi_info

def change_tempo(msg_non_note, mood):
    tempo_ratio = 1
    if mood == MOOD_CALM:
        tempo_ratio = 1.05
    elif mood == MOOD_HAPPY:
        tempo_ratio = 0.9
    elif mood == MOOD_SAD:
        tempo_ratio = 1.1
    for msg_idx in range(msg_non_note.shape[0]):
        if msg_non_note[msg_idx, 1].is_meta:
            if msg_non_note[msg_idx, 1].type == 'set_tempo':
                msg_non_note[msg_idx, 1].tempo = (int)(msg_non_note[msg_idx, 1].tempo * tempo_ratio)

def info2midi(midi_info, msg_non_note, ticks_per_beat):

    UINT32_MAX = 4294967296 - 1

    channels_block = np.empty((MAX_NUM_OF_CHANNELS), dtype=np.object)
    for i in range(MAX_NUM_OF_CHANNELS):
        channels_block[i] = np.empty((0, 5), dtype=np.uint32)

    ticks_per_beat = midi_file.ticks_per_beat

    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        for bar_idx in range(midi_info.shape[0]):
            bar_tick = ticks_per_beat * bars_beat[bar_idx]
            for info in midi_info[bar_idx, channel_idx]:
                if info[6] == 0:
                    channels_block[channel_idx] = np.vstack((channels_block[channel_idx], [(bar_idx + info[2]) * bar_tick, 0, channel_idx, info[0], info[1]]))
                    channels_block[channel_idx] = np.vstack((channels_block[channel_idx], [(bar_idx + info[2] + info[3]) * bar_tick, 1, channel_idx, info[0], info[1]]))
                else:
                    if info[2] == 0:
                        channels_block[channel_idx] = np.vstack((channels_block[channel_idx], [(bar_idx + info[2] + info[3]) * bar_tick, 1, channel_idx, info[0], info[1]]))
                    else:
                        channels_block[channel_idx] = np.vstack((channels_block[channel_idx], [(bar_idx + info[2]) * bar_tick, 0, channel_idx, info[0], info[1]]))
            channels_block[channel_idx] = channels_block[channel_idx][channels_block[channel_idx][:,0].argsort()]
    
#     with np.printoptions(precision=3, suppress=True):
#         print(channels_block[1])
            
    first_msg = np.zeros((17, 2), dtype=np.uint32)
    # initialize the comparison array
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if channels_block[channel_idx].shape[0] != 0:
            first_msg[channel_idx, 1] = channels_block[channel_idx][0, 0]
        else:
            first_msg[channel_idx, 1] = UINT32_MAX
    if msg_non_note.shape[0] != 0:
        first_msg[16, 1] = msg_non_note[0, 0]
    else:
        first_msg[16, 1] = UINT32_MAX


    midi_file_new = mido.MidiFile(type=0)
    midi_file_new.ticks_per_beat = midi_file.ticks_per_beat
    track_new = mido.MidiTrack()
    midi_file_new.tracks.append(track_new)

    total_ticks = 0

    while True:
        channel_to_append = np.argmin(first_msg[:, 1])
        if first_msg[channel_to_append, 1] == UINT32_MAX:
            break
        if channel_to_append < 16:
            info = channels_block[channel_to_append][first_msg[channel_to_append, 0]]

            tick = (int)(info[0])
            msg = mido.Message('note_on', channel=(int)(info[2]), note=(int)(info[3]), velocity=(int)(info[4]), time=0)
            if info[1] == 1:
                msg = mido.Message('note_off', channel=(int)(info[2]), note=(int)(info[3]), velocity=(int)(info[4]), time=0)

            if tick != total_ticks:
                msg.time = tick - total_ticks
                total_ticks = tick

            track_new.append(msg)

            first_msg[channel_to_append, 0] += 1
            if first_msg[channel_to_append, 0] < channels_block[channel_to_append].shape[0]:
                first_msg[channel_to_append, 1] = channels_block[channel_to_append][first_msg[channel_to_append, 0], 0]
            else:
                first_msg[channel_to_append, 1] = UINT32_MAX
        else:
            info = msg_non_note[first_msg[channel_to_append, 0]]

            tick = (int)(info[0])        
            msg = info[1]
            msg.time = 0

            if tick != total_ticks:
                msg.time = tick - total_ticks
                total_ticks = tick

            track_new.append(msg)

            first_msg[channel_to_append, 0] += 1
            if first_msg[channel_to_append, 0] < msg_non_note.shape[0]:
                first_msg[channel_to_append, 1] = msg_non_note[first_msg[channel_to_append, 0], 0]
            else:
                first_msg[channel_to_append, 1] = UINT32_MAX
    return midi_file_new
            

In [8]:
# input the file name
midi_string = '1'
mood_input = 0
midi_file = mido.MidiFile('../sample_audio/%s.mid' % midi_string)


# get the beat and tempo in each bar
_, bars_tick, bars_beat, bars_tempo = get_bars_info(midi_file)

# separate notes to their relative bars
bars_note_info = get_bars_note_info(midi_file)

# generate the rhythm pattern[channel index, bar index]
rhythm_pattern = get_rhythm_pattern(bars_note_info)

midi_info = get_midi_info(bars_note_info)

chords_predict = get_chords(midi_info)

key_predict, key_acc, chords_stat = get_key(chords_predict)

msg_non_note = get_msg_non_note(midi_file)

channels_type = get_channels_type(midi_info, chords_predict)

rhythm_start_stats, rhythm_duration_stats = get_rhythm_stats(midi_info, channels_type)
rhythm_start_recommends, rhythm_duration_recommends, rhythm_min_units = get_rhythm_recommends(rhythm_start_stats, rhythm_duration_stats)

rhythm_change(mood_input, midi_info, chords_predict, key_predict, bars_beat, channels_type, rhythm_start_recommends, rhythm_duration_recommends, rhythm_min_units)
    
add_pitch(midi_info, mood_input, key_predict, chords_predict)

change_dynamics(midi_info, mood_input, channels_type)

new_key = get_key_to_module(key_predict, mood_input)
module_vector = get_module_vector(key_predict, new_key)
module_key(midi_info, module_vector)

change_tempo(msg_non_note, mood_input)

midi_file_new = info2midi(midi_info, msg_non_note, midi_file.ticks_per_beat)

midi_file_new.save('../sample_audio_export/%s_%s.mid' % (midi_string, MOODS_NAME[mood_input]))

print("Done")

Done


In [18]:
# input the file name
midi_string = 'sample1'
midi_file = mido.MidiFile('../sample_audio/%s.mid' % midi_string)

mood_input = 3

# get the beat and tempo in each bar
_, bars_tick, bars_beat, bars_tempo = get_bars_info(midi_file)

# separate notes to their relative bars
bars_note_info = get_bars_note_info(midi_file)

# generate the rhythm pattern[channel index, bar index]
rhythm_pattern = get_rhythm_pattern(bars_note_info)

midi_info = get_midi_info(bars_note_info)

chords_predict = get_chords(midi_info)

key_predict, key_acc, chords_stat = get_key(chords_predict)

msg_non_note = get_msg_non_note(midi_file)

channels_type = get_channels_type(midi_info, chords_predict)

rhythm_start_stats, rhythm_duration_stats = get_rhythm_stats(midi_info, channels_type)
rhythm_start_recommends, rhythm_duration_recommends, rhythm_min_units = get_rhythm_recommends(rhythm_start_stats, rhythm_duration_stats)

rhythm_change(mood_input, midi_info, chords_predict, key_predict, bars_beat, channels_type, rhythm_start_recommends, rhythm_duration_recommends, rhythm_min_units)
    
add_pitch(midi_info, mood_input, key_predict, chords_predict)

# with np.printoptions(precision=3, suppress=True):
#     print(midi_info[18:22, 5])
    
change_dynamics(midi_info, mood_input, channels_type)

# with np.printoptions(precision=3, suppress=True):
#     print(midi_info[18:22, 5])

new_key = get_key_to_module(key_predict, mood_input)

module_vector = get_module_vector(key_predict, new_key)
module_key(midi_info, module_vector)

change_tempo(msg_non_note, mood_input)

# new_chords_predict = get_chords(midi_info)

# new_key_predict, new_key_acc, new_chords_stat = get_key(new_chords_predict)

midi_file_new = info2midi(midi_info, msg_non_note, midi_file.ticks_per_beat)

midi_file_new.save('../sample_audio_export/%s_%s.mid' % (midi_string, MOODS_NAME[mood_input]))

with np.printoptions(precision=3, suppress=True):
    print('Key: %s (accuracy: %f)' % (name_key(key_predict), key_acc))
#     print('\nChord statistics')
#     for chord_stat_idx, chord_stat in enumerate(chords_stat):
#         if chord_stat != 0:
#             print('%s\t: %d' % (name_chord(chord_stat_idx), chord_stat))
#     print('New Key Destination: %s' % name_key(new_key))
#     print('New Key: %s (accuracy: %f)' % (name_key(new_key_predict), new_key_acc))
#     print('\nChord statistics')
#     for chord_stat_idx, chord_stat in enumerate(new_chords_stat):
#         if chord_stat != 0:
#             print('%s\t: %d' % (name_chord(chord_stat_idx), chord_stat))
#     print([name_chord(chord) for chord in key_to_chords(key_predict)[0]])
#     print([name_chord(chord) for chord in key_to_chords(new_key)[0]])
#     print(rhythm_duration_stats)
#     print(rhythm_min_units)
#     print(channels_type)

pad dynamic: [85]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 90 87]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 90 87]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 85 80 77]
pad dynamic: [95 95 90 87]
pad dynamic: [95 95 85 80 77]
p

C:\Users\franc\Anaconda3\envs\mido\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\franc\Anaconda3\envs\mido\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


lead dynamic: [ 68  82 127  49  99  88  93]
lead dynamic: [ 68  82 127  49  99  88  93]
lead dynamic: [ 68  85 127  49 127  82]
lead dynamic: [ 68  85 112  49 127  98]
lead dynamic: []
lead dynamic: []
lead dynamic: [ 0 49 60]
lead dynamic: [100 127  49]
lead dynamic: [ 0 49]
lead dynamic: [0]
lead dynamic: [ 0 66 48]
lead dynamic: [100 117 127  49]
lead dynamic: [ 0 49]
lead dynamic: [0]
lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: []
pad dynamic: [106 103 101  99]
pad dynamic: [106 106  98  96]
pad dynamic: [104 103  99]
pad dynamic: [106 106 103  98]
pad dynamic: [103 10

lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60]
lead dynamic: [60 60 60 60 60 60 60]
lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic: []
lead dynamic:

In [300]:
midi_file_new = info2midi(midi_info, msg_non_note, midi_file.ticks_per_beat)

midi_file_new.save('../sample_audio_export/%s_%s.mid' % (midi_string, MOODS_NAME[mood_input]))

import time    

# use 'print(mido.get_output_names())' to get your port's name and type it in the 'mido.open_output' bracket
port = None
port = mido.open_output('Microsoft GS Wavetable Synth 0')
play_audio = False

try:

    for msg in midi_file_new:
        
        # delay for playing
        if play_audio:
            time.sleep(msg.time)
            if not msg.is_meta:
                port.send(msg)
#                 print(msg)

    if port is not None:
        port.close()
    print('End of midi')
    
# action after pressing stop
except KeyboardInterrupt:
    
    if port is not None:
        port.close()
    print('Forced stop')

End of midi


In [87]:
# def get_rhythm_stats(midi_info, channels_type):
#     rhythm_start_stats = np.empty((MAX_NUM_OF_CHANNELS), dtype=np.object)
#     rhythm_duration_stats = np.empty((MAX_NUM_OF_CHANNELS), dtype=np.object)
#     for channel_idx in range(MAX_NUM_OF_CHANNELS):
#         if channels_type[channel_idx] != CHANNEL_TYPE_EMPTY:
#             rhythm_start_stat = np.empty((0, 2))
#             rhythm_duration_stat = np.empty((0, 2))
#             for bar_idx in range(midi_info.shape[0]):
#                 for note_info in midi_info[bar_idx, channel_idx]:
#                     if rhythm_start_stat.shape[0] == 0:
#                         rhythm_start_stat = np.vstack((rhythm_start_stat, [note_info[4], 1]))
#                         rhythm_duration_stat = np.vstack((rhythm_duration_stat, [note_info[5], 1]))
#                     else:
#                         if note_info[4] in rhythm_start_stat[:, 0]:
#                             rhythm_start_stat[np.argwhere(rhythm_start_stat[:, 0] == note_info[4])[0][0], 1] += 1
#                         else:
#                             rhythm_start_stat = np.vstack((rhythm_start_stat, [note_info[4], 1]))
#                         if note_info[5] in rhythm_duration_stat[:, 0]:
#                             rhythm_duration_stat[np.argwhere(rhythm_duration_stat[:, 0] == note_info[5])[0][0], 1] += 1
#                         else:
#                             rhythm_duration_stat = np.vstack((rhythm_duration_stat, [note_info[5], 1]))

#             rhythm_start_stats[channel_idx] = rhythm_start_stat[rhythm_start_stat[:, 1] >= np.sum(rhythm_start_stat[:, 1]) * 0.05]
#             rhythm_duration_stats[channel_idx] = rhythm_duration_stat[rhythm_duration_stat[:, 1] >= np.sum(rhythm_duration_stat[:, 1]) * 0.025]
#     return rhythm_start_stats, rhythm_duration_stats
            
# RHYTHM_UNITS = np.array([
#     1,
#     1 / 2,
#     1 / 3,
#     1 / 4,
#     1 / 6,
#     1 / 8,
#     1 / 12,
#     1 / 16,
#     1 / 24,
#     1 / 32,
# ])

# def get_rhythm_recommends(rhythm_start_stats, rhythm_duration_stats):
#     rhythm_start_recommends = np.empty((MAX_NUM_OF_CHANNELS), dtype=np.object)
#     for channel_idx in range(MAX_NUM_OF_CHANNELS):
#         if rhythm_start_stats[channel_idx] is None:
#             continue
#         unit = 0
#         unit_min = 1
#         unit_val = 12345
#         for unit in RHYTHM_UNITS:
#             if np.sum(np.mod(rhythm_start_stats[channel_idx][:, 0] / unit, 1)) < unit:
#                 unit_min = -1
#                 break
#             else:
#                 if np.count_nonzero(np.mod(rhythm_start_stats[channel_idx][:, 0] / unit, 1)) < unit_val:
#                     unit_min = unit
#                     unit_val = np.count_nonzero(np.mod(rhythm_start_stats[channel_idx][:, 0] / unit, 1))
#         if unit_min != -1:
#             unit = unit_min
#         beat = np.ceil(np.max(rhythm_start_stats[channel_idx][:, 0]))
#         rhythm_start_recommends[channel_idx] = np.linspace(0, beat, num=(int)(beat / unit), endpoint=False)

#     rhythm_duration_recommends = np.empty((MAX_NUM_OF_CHANNELS), dtype=np.object)
#     for channel_idx in range(MAX_NUM_OF_CHANNELS):
#         if rhythm_duration_stats[channel_idx] is None:
#             continue

#         offset_min = 100
#         for rhythm_duration_stat in rhythm_duration_stats[channel_idx]:
#             for rhythm_start_recommend in rhythm_start_recommends[channel_idx]:
#                 if rhythm_start_recommend - rhythm_duration_stat[0] >= 0:
#                     if offset_min > (rhythm_start_recommend - rhythm_duration_stat[0]) and (rhythm_start_recommend - rhythm_duration_stat[0]) > 0:
#                         offset_min = rhythm_start_recommend - rhythm_duration_stat[0]
#                     break
#         if offset_min == 100:
#             offset_min = 0

#         rhythm_duration_recommend = np.array([])
#         for rhythm_start_recommend in rhythm_start_recommends[channel_idx]:
#             duration_recommend = rhythm_start_recommend / 2 - offset_min
#             if duration_recommend > 0:
#                 rhythm_duration_recommend = np.hstack((rhythm_duration_recommend, duration_recommend))
#             duration_recommend = rhythm_start_recommend - offset_min
#             if duration_recommend > 0:
#                 rhythm_duration_recommend = np.hstack((rhythm_duration_recommend, duration_recommend))
#         rhythm_duration_recommends[channel_idx] = rhythm_duration_recommend
#     return rhythm_start_recommends, rhythm_duration_recommends
                

# rhythm_start_stats, rhythm_duration_stats = get_rhythm_stats(midi_info, channels_type)
# rhythm_start_recommends, rhythm_duration_recommends = get_rhythm_recommends(rhythm_start_stats, rhythm_duration_stats)
# # with np.printoptions(precision=3, suppress=True):
# #     print(rhythm_start_recommends)
# #     print(rhythm_start_stats)
# #     print(rhythm_duration_recommends)
# #     print(rhythm_duration_stats)

TypeError: 'NoneType' object is not iterable

In [5]:
# keys notes, '[C, Db, D, ..., B, Cm, Dbm, ..., Bbm, Bm]'
keys_notes = np.empty((0, 12), dtype=np.int8)

keys_notes_entries = np.array([
    # C Major
    [2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    # C minor
    [2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1],
], dtype=np.int8)

for keys_notes_entry in keys_notes_entries:
    for i in range(12):
        keys_notes = np.vstack((keys_notes, np.roll(keys_notes_entry[0:12], i)))

# chords notes, '[C, Db, D, ..., B, Cm, Dbm, ..., Bbm, Bm]'
chords_notes = np.empty((0, 12), dtype=np.int8)

chords_notes_entries = np.array([
    # C1
    [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    # C5
    [2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    # C
    [2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    # Cm
    [2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1],
    # C7
    [2, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0],
    # CMaj7
    [2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    # Cmin7
    [2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0],
    # Cdim
    [2, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
], dtype=np.int8)

for chords_notes_entry in chords_notes_entries:
    for i in range(12):
        chords_notes = np.vstack((chords_notes, np.roll(chords_notes_entry, i)))
        
# get each note scores in a bar based on only its note (higher score = more related to the key)
def get_bar_notes_priority(bar_info, key_predict, chord_predict):
    notes_score = keys_notes[key_predict] + chords_notes[chord_predict]
    bar_notes_priority = np.zeros(bar_info.shape[0], dtype=np.uint8)
    for idx, note_info in enumerate(bar_info):
        bar_notes_priority[idx] = notes_score[(int)(note_info[0] % 12)]
    return bar_notes_priority

# example
example_key_predict = 14
example_chord_predict = 57
example_bar_info = np.array([
    [41, 88, 0, 0.184, 0, 0.738, 0],
    [53, 88, 0.188, 0.059, 0.75, 0.237, 0],
    [41, 88, 0.25, 0.184, 1, 0.738, 0],
    [53, 88, 0.438, 0.059, 1.75, 0.237, 0],
    [38, 88, 0.5, 0.184, 2, 0.738, 0],
    [50, 88, 0.688, 0.059, 2.75, 0.237, 0],
    [38, 88, 0.75, 0.184, 3, 0.738, 0],
    [50, 88, 0.938, 0.059, 3.75, 0.237, 0],
])
with np.printoptions(precision=3, suppress=True):
    print("key:\t%s" % name_key(example_key_predict))
    print("bar:\t%s" % name_chord(example_chord_predict))
    print("note:\t%s" % example_bar_info[:, 0])
    print("prior:\t%s" % get_bar_notes_priority(example_bar_info, example_key_predict, example_chord_predict))
    print(example_bar_info)

key:	D minor
bar:	A7
note:	[41. 53. 41. 53. 38. 50. 38. 50.]
prior:	[1 1 1 1 3 3 3 3]
[[41.    88.     0.     0.184  0.     0.738  0.   ]
 [53.    88.     0.188  0.059  0.75   0.237  0.   ]
 [41.    88.     0.25   0.184  1.     0.738  0.   ]
 [53.    88.     0.438  0.059  1.75   0.237  0.   ]
 [38.    88.     0.5    0.184  2.     0.738  0.   ]
 [50.    88.     0.688  0.059  2.75   0.237  0.   ]
 [38.    88.     0.75   0.184  3.     0.738  0.   ]
 [50.    88.     0.938  0.059  3.75   0.237  0.   ]]


In [6]:
# add pitch to notes with -1 pitch
# analysis by "median note over bar", "key" and "bar chord"
def bar_add_pitch(bar_info, key_predict, chord_predict):
    avg_pitch = np.median([note for note in bar_info[:, 0] if note != -1])
    notes_score = keys_notes[key_predict] + chords_notes[chord_predict]
    print("notes score:\t%s" % notes_score)

    SEARCH_PITCH_RANGE = 12
    min_pitch = (int)(np.max((0, np.min((np.floor(avg_pitch - SEARCH_PITCH_RANGE), np.min([note for note in bar_info[:, 0] if note != -1]))))))
    max_pitch = (int)(np.min((127, np.max((np.floor(avg_pitch + SEARCH_PITCH_RANGE), np.max(bar_info[:, 0]))))))

    notes_priority = np.zeros(max_pitch - min_pitch + 1)
    for i in range(0, max_pitch - min_pitch + 1):
        notes_priority[i] = 1 / (np.abs(avg_pitch - (i + min_pitch)) + SEARCH_PITCH_RANGE) * notes_score[(i + min_pitch) % 12]

    print(notes_priority)
        
    avail_notes = np.argsort(-notes_priority) + min_pitch
    print(avail_notes)

    for empty_idx in np.argwhere(bar_info[:, 0] == -1).flatten():
        notes_on = []
        for note_info in [note_info for note_info in bar_info if note_info[0] != -1]:
            bool_overlap = not(note_info[2] >= (bar_info[empty_idx, 2] + bar_info[empty_idx, 3]) or (note_info[2] + note_info[3]) <= bar_info[empty_idx, 2])
            if (note_info[0] not in notes_on and bool_overlap):
                notes_on.append(note_info[0])
        i = 0
        print(notes_on)
        while (avail_notes[i] in notes_on):
            i += 1
            if i == avail_notes.shape[0]:
                i = 0
                break
        bar_info[empty_idx, 0] = avail_notes[i]
        
    return bar_info
    
# real application
for bar_idx in range(midi_info.shape[0]):
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if (-1 in midi_info[bar_idx, channel_idx][:, 0]):
            midi_info[bar_idx, channel_idx] = bar_add_pitch(midi_info[bar_idx, channel_idx], key_predict, chord_predict[bar_idx])

# example: fill -1 pitch
# create five notes with -1 pitch
example_key_predict = 14
example_chord_predict = 57
example_bar_info = np.array([
    [41, 88, 0, 0.184, 0, 0.738, 0],
    [-1, 88, 0, 0.184, 0, 0.738, 0],
    [-1, 88, 0, 0.184, 0, 0.738, 0],
    [-1, 88, 0, 0.184, 0, 0.738, 0],
    [-1, 88, 0, 0.184, 0, 0.738, 0],
    [53, 88, 0.188, 0.059, 0.75, 0.237, 0],
    [41, 88, 0.25, 0.184, 1, 0.738, 0],
    [-1, 88, 0.25, 0.75, 1, 0.738, 0],
    [53, 88, 0.438, 0.059, 1.75, 0.237, 0],
    [38, 88, 0.5, 0.184, 2, 0.738, 0],
    [50, 88, 0.688, 0.059, 2.75, 0.237, 0],
    [38, 88, 0.75, 0.184, 3, 0.738, 0],
    [50, 88, 0.938, 0.059, 3.75, 0.237, 0],
])
with np.printoptions(precision=3, suppress=True):
    print("key:\t%s" % name_key(example_key_predict))
    print("bar:\t%s" % name_chord(example_chord_predict))
#     print("before:\t%s" % example_bar_info[:, 0])
    print("after:\t%s" % bar_add_pitch(example_bar_info, example_key_predict, example_chord_predict)[:, 0])
    print(example_bar_info)

key:	D minor
bar:	A7
notes score:	[2 2 3 0 2 1 0 2 0 3 1 1]
[0.122 0.043 0.044 0.093 0.098 0.154 0.    0.114 0.061 0.    0.138 0.
 0.24  0.08  0.074 0.138 0.129 0.182 0.    0.108 0.051 0.    0.093 0.
 0.128]
[45 50 38 48 43 49 57 33 40 52 37 55 36 46 47 41 53 35 34 42 44 56 39 51
 54]
[41.0]
[41.0, 45.0]
[41.0, 45.0, 50.0]
[41.0, 45.0, 50.0, 38.0]
[41.0, 53.0, 38.0, 50.0]
after:	[41. 45. 50. 38. 48. 53. 41. 45. 53. 38. 50. 38. 50.]
[[41.    88.     0.     0.184  0.     0.738  0.   ]
 [45.    88.     0.     0.184  0.     0.738  0.   ]
 [50.    88.     0.     0.184  0.     0.738  0.   ]
 [38.    88.     0.     0.184  0.     0.738  0.   ]
 [48.    88.     0.     0.184  0.     0.738  0.   ]
 [53.    88.     0.188  0.059  0.75   0.237  0.   ]
 [41.    88.     0.25   0.184  1.     0.738  0.   ]
 [45.    88.     0.25   0.75   1.     0.738  0.   ]
 [53.    88.     0.438  0.059  1.75   0.237  0.   ]
 [38.    88.     0.5    0.184  2.     0.738  0.   ]
 [50.    88.     0.688  0.059  2.75   0.237 

In [7]:
# get the overall statistic of dynamics in each channel ([min, max, median])
def get_dynamics_info(midi_info):
    dynamics_info = np.zeros((MAX_NUM_OF_CHANNELS, 4), dtype=np.int16)
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        notes_dynamics = []
        for bar_idx in range(midi_info.shape[0]):
            for bar_info in midi_info[bar_idx, channel_idx]:
                notes_dynamics.append(bar_info[1])
        if len(notes_dynamics) > 0:
            dynamics_info[channel_idx] = [np.min(notes_dynamics), np.max(notes_dynamics), np.median(notes_dynamics), np.std(notes_dynamics)]
    return dynamics_info

def get_dynamics_lookup_table(dynamics_info, wide_factor=1, avg_addition_boost=0, avg_multiply_boost=1):
    dynamics_lookup_table = np.zeros((MAX_NUM_OF_CHANNELS, 128), dtype=np.int16)
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        dmin = dynamics_info[channel_idx, 0]
        dmax = dynamics_info[channel_idx, 1]
        dmed = dynamics_info[channel_idx, 2]
        for i in range(dmin, dmax + 1):
            note_velocity = 0
            if i < dmed:
                note_velocity = (int)(dmed - (dmed - dmin) * np.power((dmed - i) / (dmed - dmin), wide_factor) * avg_multiply_boost)
            else:
                if dmax == dmed:
                    note_velocity = i
                else:
                    note_velocity = (int)(dmed - (dmed - dmax) * np.power((dmed - i) / (dmed - dmax), wide_factor) * avg_multiply_boost)
            dynamics_lookup_table[channel_idx, i] = np.max((0, np.min((127, note_velocity + avg_addition_boost))))
    return dynamics_lookup_table

dynamics_info = get_dynamics_info(midi_info)
# print(dynamics_info)
dynamics_lookup_table = get_dynamics_lookup_table(dynamics_info, 2, 10)
# print(dynamics_lookup_table[5,:])
 

def get_dynamics_angry_factors(dynamics_info, wide_para_1=0.3, wide_para_2=0.1, addition_para_1=10, multiply_para_1=0.2, multiply_para_2=0):
    dynaics_angry_factors = np.zeros((MAX_NUM_OF_CHANNELS, 3))
    dynaics_angry_factors[:, 0] = 1 - 1 / (dynamics_info[:, 3] * wide_para_1 + wide_para_2 + 1)
    dynaics_angry_factors[:, 1] = np.repeat(addition_para_1, MAX_NUM_OF_CHANNELS)
    dynaics_angry_factors[:, 2] = 1 + multiply_para_1 / (dynamics_info[:, 3] + multiply_para_2 + 1)
    return dynaics_angry_factors

def get_dynamics_calm_factors(dynamics_info, wide_para_1=0.8, addition_para_1=-10, multiply_para_1=0.2, multiply_para_2=0.5):
    dynaics_calm_factors = np.zeros((MAX_NUM_OF_CHANNELS, 3))
    dynaics_calm_factors[:, 0] = 1 + dynamics_info[:, 3] * wide_para_1
    dynaics_calm_factors[:, 1] = np.repeat(addition_para_1, MAX_NUM_OF_CHANNELS)
    dynaics_calm_factors[:, 2] = 1 / (dynamics_info[:, 3] * multiply_para_1 + 1) * (1 - multiply_para_2) + multiply_para_2
    return dynaics_calm_factors
    
def get_dynamics_happy_factors(dynamics_info, wide_para_1=5, wide_para_2=0.8, wide_para_3=0.5, addition_para_1=0, multiply_para_1=0.2, multiply_para_2=0):
    dynamics_happy_factors = np.zeros((MAX_NUM_OF_CHANNELS, 3))
    for i in range(MAX_NUM_OF_CHANNELS):
        if dynamics_info[i, 3] < wide_para_1:
            dynamics_happy_factors[i, 0] = 1 - wide_para_2 * np.power((wide_para_1 - dynamics_info[i, 3]) / wide_para_1, wide_para_2)
        else:
            dynamics_happy_factors[i, 0] = (1 - 1 / ((dynamics_info[i, 3] - wide_para_1) * wide_para_3 + 1)) * wide_para_2 + 1
    dynamics_happy_factors[:, 0] = 1 - 1 / (dynamics_info[:, 3] * wide_para_1 + wide_para_2 + 1)
    dynamics_happy_factors[:, 1] = np.repeat(addition_para_1, MAX_NUM_OF_CHANNELS)
    dynamics_happy_factors[:, 2] = 2 / (dynamics_info[:, 3] + 5) + 0.8
    return dynamics_happy_factors

def dynamics_angry(bar_info, dynamic_info, angry_dynamic_lookup_table, bar_notes_priority):
    if len(bar_info) == 0:
        return np.array([])    
    if np.min(bar_info[:,1]) != np.max(bar_info[:,1]) or len(bar_info) == 1:
        return angry_dynamic_lookup_table[[(int)(velocity) for velocity in bar_info[:, 1]]]
    else:
        dynamics_new = np.array([
            np.round(dynamic_info[0] + (dynamic_info[1] - dynamic_info[0]) / 5),
            np.round(dynamic_info[0] + (dynamic_info[1] - dynamic_info[0]) / 5 * 2),
            np.round(dynamic_info[0] + (dynamic_info[1] - dynamic_info[0]) / 5 * 3),
            np.round(dynamic_info[0] + (dynamic_info[1] - dynamic_info[0]) / 5 * 4)
        ], dtype=np.int8)
#         print(dynamics_new)
        return angry_dynamic_lookup_table[dynamics_new[bar_notes_priority]]

def dynamics_calm(bar_info, calm_dynamic_lookup_table):
    return calm_dynamic_lookup_table[[(int)(velocity) for velocity in bar_info[:, 1]]]

def dynamics_happy(bar_info, happy_dynamic_lookup_table):
    return happy_dynamic_lookup_table[[(int)(velocity) for velocity in bar_info[:, 1]]]
    
# example
example_key_predict = 14
example_chord_predict = 57
example_bar_info_1 = np.array([
    [41, 90, 0, 0.184, 0, 0.738, 0],
    [53, 70, 0.188, 0.059, 0.75, 0.237, 0],
    [41, 80, 0.25, 0.184, 1, 0.738, 0],
    [53, 60, 0.438, 0.059, 1.75, 0.237, 0],
    [38, 90, 0.5, 0.184, 2, 0.738, 0],
    [50, 70, 0.688, 0.059, 2.75, 0.237, 0],
    [38, 80, 0.75, 0.184, 3, 0.738, 0],
    [50, 60, 0.938, 0.059, 3.75, 0.237, 0],
])
example_bar_info_2 = np.array([
    [41, 88, 0, 0.184, 0, 0.738, 0],
    [53, 88, 0.188, 0.059, 0.75, 0.237, 0],
    [41, 88, 0.25, 0.184, 1, 0.738, 0],
    [53, 88, 0.438, 0.059, 1.75, 0.237, 0],
    [38, 88, 0.5, 0.184, 2, 0.738, 0],
    [50, 88, 0.688, 0.059, 2.75, 0.237, 0],
    [38, 88, 0.75, 0.184, 3, 0.738, 0],
    [50, 88, 0.938, 0.059, 3.75, 0.237, 0],
])
example_dynamic_info = np.array([60, 100, 88, 15])
example_dynamics_info = np.tile(example_dynamic_info, (16, 1))
example_bar_notes_priority = get_bar_notes_priority(example_bar_info, example_key_predict, example_chord_predict)
example_angry_dynamic_factors = get_dynamics_angry_factors(example_dynamics_info)[0]
example_calm_dynamic_factors = get_dynamics_calm_factors(example_dynamics_info)[0]
example_happy_dynamic_factors = get_dynamics_happy_factors(example_dynamics_info)[0]
example_angry_dynamic_lookup_table = get_dynamics_lookup_table(example_dynamics_info, wide_factor=example_angry_dynamic_factors[0], avg_addition_boost=example_angry_dynamic_factors[1], avg_multiply_boost=example_angry_dynamic_factors[2])[0]
example_calm_dynamic_lookup_table = get_dynamics_lookup_table(example_dynamics_info, wide_factor=example_calm_dynamic_factors[0], avg_addition_boost=example_calm_dynamic_factors[1], avg_multiply_boost=example_calm_dynamic_factors[2])[0]
example_happy_dynamic_lookup_table = get_dynamics_lookup_table(example_dynamics_info, wide_factor=example_happy_dynamic_factors[0], avg_addition_boost=example_happy_dynamic_factors[1], avg_multiply_boost=example_happy_dynamic_factors[2])[0]
example_dynamics_angry_1 = dynamics_angry(example_bar_info_1, example_dynamic_info, example_angry_dynamic_lookup_table, example_bar_notes_priority)
example_dynamics_angry_2 = dynamics_angry(example_bar_info_2, example_dynamic_info, example_angry_dynamic_lookup_table, example_bar_notes_priority)
example_dynamics_calm_1 = dynamics_calm(example_bar_info_1, example_calm_dynamic_lookup_table)
example_dynamics_calm_2 = dynamics_calm(example_bar_info_2, example_calm_dynamic_lookup_table)
example_dynamics_happy_1 = dynamics_happy(example_bar_info_1, example_happy_dynamic_lookup_table)
with np.printoptions(precision=3, suppress=True):
    print("angry dynamic lookup table:\n%s" % example_angry_dynamic_lookup_table)
    print("calm dynamic lookup table:\n%s" % example_calm_dynamic_lookup_table)
    print("happy dynamic lookup table:\n%s" % example_happy_dynamic_lookup_table)
    print("ex.1 before:\t%s" % example_bar_info_1[:, 1])
#     print("angry1 after:\t%s" % example_dynamics_angry_1)
    print("calm1 after:\t%s" % example_dynamics_calm_1)
    print("ex.2 before:\t%s" % example_bar_info_2[:, 1])
    print("angry2 after:\t%s" % example_dynamics_angry_2)
    print("calm2 after:\t%s" % example_dynamics_calm_2)
    print("ex.1 before:\t%s" % example_bar_info_1[:, 1])
    print("happy1 after:\t%s" % example_dynamics_happy_1)
    
    print("before:\n%s" % example_bar_info_1)
    temp = example_bar_info_1
    temp[:, 1] = example_dynamics_angry_1
    print("after:\n%s" % temp)
    print(dynamics_info)
    print(get_dynamics_lookup_table(example_dynamics_info, wide_factor=2, avg_addition_boost= 10, avg_multiply_boost=1.5)[0])
    

angry dynamic lookup table:
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0  69  70  71  72  73  73  74  75  76  77  78  79
  80  81  81  82  83  84  85  86  87  88  90  91  92  93  94  96  98  99
 100 101 102 103 104 105 106 107 108 109 110   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
calm dynamic lookup table:
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0 60 67 71 73 75 76 77 77 77 77 77 77
 77 77 77 77 77 77 77 77 77 77 77 77 77 77 77 78 78 78 78 78 78 78 78 78
 78 78 78 80 85  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
happy dynamic lookup table:
[ 0  

In [8]:
# chords element, '[C, Db, D, ..., B, Cm, Dbm, ..., Bbm, Bm]'
chords_elements = np.empty((0, 12), dtype=np.int8)

chords_elements_entries = np.array([
    # C1
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    # C5
    [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    # C
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    # Cm
    [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    # C7
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
    # CMaj7
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    # Cmin7
    [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    # Cdim
    [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
], dtype=np.int8)

for chords_elements_entry in chords_elements_entries:
    for i in range(12):
        chords_elements = np.vstack((chords_elements, np.roll(chords_elements_entry, i)))

def is_note_in_chord(note, chord):
    return chords_elements[chord][note % 12] == 1
        
example_chord = 44
example_note = 51
print("Chord:\t%s" % name_chord(example_chord))
print("Note:\t%s" % note_int2char(example_note))
print ("Is in chord?\t%s" % is_note_in_chord(example_note, example_chord))

Chord:	Abm
Note:	Eb
Is in chord?	True


In [52]:
def dynamics_sad_lead(bar_info, dynamic_info):
    notes_velocity = np.zeros((bar_info.shape[0], ), dtype=np.int8)
    notes_velocity_ratio = np.zeros((bar_info.shape[0], ), dtype=np.float_)
    
    bar_med_note = np.median(bar_info[:, 1])
    for i in range(bar_info.shape[0]):
        if i == 0:
            notes_velocity_ratio[i] = 0
        else:
            notes_velocity_ratio[i] = (bar_info[i, 0] - bar_info[i - 1, 0]) / bar_info[i, 0]
    max_ratio = np.max(notes_velocity_ratio)
    min_ratio = np.min(notes_velocity_ratio)
    for i in range(bar_info.shape[0]):
        if notes_velocity_ratio[i] < 0:
            notes_velocity[i] = (int)(bar_med_note - notes_velocity_ratio[i] * (bar_med_note - dynamic_info[0]) / min_ratio)
        else:
            notes_velocity[i] = (int)(bar_med_note + notes_velocity_ratio[i] * (dynamic_info[1] - bar_med_note) / max_ratio)
    return notes_velocity
        
def dynamics_sad_pad(bar_info, dynamic_info):
    bar_med_note = np.median(bar_info[:, 1])
    notes_velocity = np.array((1 - bar_info[:, 2]) * (bar_med_note - dynamic_info[0]) + dynamic_info[0], dtype=np.int8)
    return notes_velocity
    
temp = midi_info[40, 1]        

with np.printoptions(precision=3, suppress=True):
    print(temp)
    print(dynamics_sad_lead(temp, np.array([50, 127, 88, 10])))
    print(dynamics_sad_pad(temp, np.array([50, 127, 88, 10])))

[[ 41.    112.      0.      0.092   0.      0.369   0.   ]
 [ 50.    115.      0.092   0.092   0.369   0.369   0.   ]
 [ 53.    112.      0.188   0.03    0.75    0.119   0.   ]
 [ 50.    115.      0.217   0.03    0.869   0.119   0.   ]
 [ 41.    112.      0.25    0.092   1.      0.369   0.   ]
 [ 50.    115.      0.342   0.092   1.369   0.369   0.   ]
 [ 53.    112.      0.438   0.03    1.75    0.119   0.   ]
 [ 50.    115.      0.467   0.03    1.869   0.119   0.   ]
 [ 38.    115.      0.5     0.184   2.      0.738   0.   ]
 [ 50.    115.      0.688   0.059   2.75    0.237   0.   ]
 [ 72.    112.      0.75    0.184   3.      0.738   0.   ]
 [ 74.    115.      0.75    0.184   3.      0.738   0.   ]
 [ 72.    112.      0.938   0.062   3.75    0.25    1.   ]
 [ 74.    115.      0.938   0.062   3.75    0.25    1.   ]
 [ 65.    112.      0.938   0.062   3.75    0.25    0.   ]]
[115 122 117 102  69 122 117 102  50 124 127 116 109 116  86]
[115 109 102 100  98  92  86  84  82  70  66  66  54

In [155]:
temp = np.array([
    [41, 88, 0, 0.184, 0, 0.738, 0],
    [53, 88, 0.188, 0.059, 0.75, 0.237, 0],
    [41, 88, 0.25, 0.184, 1, 0.738, 0],
    [53, 88, 0.438, 0.059, 1.75, 0.237, 0],
    [38, 88, 0.5, 0.184, 2, 0.738, 0],
    [50, 88, 0.688, 0.059, 2.75, 0.237, 0],
    [38, 88, 0.75, 0.184, 3, 0.738, 0],
    [50, 88, 0.938, 0.059, 3.75, 0.237, 0],
])
temp2 = np.array([
    [40, 88, 0, 0.184, 0, 0.738, 0],
    [52, 88, 0.188, 0.059, 0.75, 0.237, 0],
    [40, 88, 0.25, 0.184, 1, 0.738, 0],
    [52, 88, 0.438, 0.059, 1.75, 0.237, 0],
    [36, 88, 0.5, 0.184, 2, 0.738, 0],
    [48, 88, 0.688, 0.059, 2.75, 0.237, 0],
    [36, 88, 0.75, 0.184, 3, 0.738, 0],
    [48, 88, 0.938, 0.059, 3.75, 0.237, 0],
])

with np.printoptions(precision=3, suppress=True):
    print("origin: D minor\n%s" % temp)
    print([note_int2char((int)(note[0])) for note in temp])
    print("modulated: C Major\n%s" % temp2)
    print([note_int2char((int)(note[0])) for note in temp2])

origin: D minor
[[41.    88.     0.     0.184  0.     0.738  0.   ]
 [53.    88.     0.188  0.059  0.75   0.237  0.   ]
 [41.    88.     0.25   0.184  1.     0.738  0.   ]
 [53.    88.     0.438  0.059  1.75   0.237  0.   ]
 [38.    88.     0.5    0.184  2.     0.738  0.   ]
 [50.    88.     0.688  0.059  2.75   0.237  0.   ]
 [38.    88.     0.75   0.184  3.     0.738  0.   ]
 [50.    88.     0.938  0.059  3.75   0.237  0.   ]]
['F', 'F', 'F', 'F', 'D', 'D', 'D', 'D']
modulated: C Major
[[40.    88.     0.     0.184  0.     0.738  0.   ]
 [52.    88.     0.188  0.059  0.75   0.237  0.   ]
 [40.    88.     0.25   0.184  1.     0.738  0.   ]
 [52.    88.     0.438  0.059  1.75   0.237  0.   ]
 [36.    88.     0.5    0.184  2.     0.738  0.   ]
 [48.    88.     0.688  0.059  2.75   0.237  0.   ]
 [36.    88.     0.75   0.184  3.     0.738  0.   ]
 [48.    88.     0.938  0.059  3.75   0.237  0.   ]]
['E', 'E', 'E', 'E', 'C', 'C', 'C', 'C']


In [95]:
CHANNEL_TYPE_NAME = ['lead', 'pad', 'percussion', 'empty']
CHANNEL_TYPE_LEAD = 0
CHANNEL_TYPE_PAD = 1
CHANNEL_TYPE_PERCUSSION = 2
CHANNEL_TYPE_EMPTY = 3
        
def get_channels_type(midi_info, chords_predict):
    channels_inchord_count = np.zeros((MAX_NUM_OF_CHANNELS))
    channels_all_count = np.zeros((MAX_NUM_OF_CHANNELS))
    channels_chord_acc = np.zeros((MAX_NUM_OF_CHANNELS))
    for bar_idx in range(midi_info.shape[0]):
        for channel_idx in range(MAX_NUM_OF_CHANNELS):
            for note_info in midi_info[bar_idx, channel_idx]:
                if is_note_in_chord(note_info[0], chords_predict[bar_idx]):
                    channels_inchord_count[channel_idx] += note_info[3]
                channels_all_count[channel_idx] += note_info[3]
    
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if channels_all_count[channel_idx] != 0:
            channels_chord_acc[channel_idx] = channels_inchord_count[channel_idx] / channels_all_count[channel_idx]
    
    channels_type = np.zeros((MAX_NUM_OF_CHANNELS), dtype=np.int8)
    inchord_count = 0
    all_count = 0
    chord_acc = 0
    
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if channels_chord_acc[channel_idx] == 0:
            channels_type[channel_idx] = CHANNEL_TYPE_EMPTY
        elif channels_chord_acc[channel_idx] <= 0.5:
            channels_type[channel_idx] = CHANNEL_TYPE_PERCUSSION
        else:
            inchord_count += channels_inchord_count[channel_idx]
            all_count += channels_all_count[channel_idx]
    chord_acc = inchord_count / all_count
    
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if channels_type[channel_idx] == 0:
            if channels_chord_acc[channel_idx] < chord_acc:
                channels_type[channel_idx] = CHANNEL_TYPE_LEAD
            else:
                channels_type[channel_idx] = CHANNEL_TYPE_PAD
    return channels_type

with np.printoptions(precision=3, suppress=True):
#     print(midi_info[bar_idx, :])
#     print(bar_stat)
    print(get_channels_type(midi_info, chords_predict))

0.8738252982331222
[0.939 0.821 0.943 0.869 0.859 0.795 0.868 0.946 0.82  0.346 0.847 0.812
 0.    0.    0.    0.   ]
[1 0 1 0 0 0 0 1 0 2 0 0 3 3 3 3]


In [391]:
for msg_row in msg_non_note:
    msg = msg_row[1]
    if msg.is_meta:
        if msg.type == 'set_tempo':
            msg.tempo = 1000

[[0 <meta message track_name name='01-muse_2009-uprising-[k]' time=0>]
 [0 <meta message set_tempo tempo=1000 time=0>]
 [0
  <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>]
 ...
 [309549.0 <message pitchwheel channel=1 pitch=-4736 time=52>]
 [309600.0 <message pitchwheel channel=1 pitch=0 time=51>]
 [309600.0 <meta message end_of_track time=0>]]
